Ch 1-10: Array, Strings, Lists, Stacks, Queues, Trees, Graphs, DP, Hard Problems -- [Open in Colab](https://colab.research.google.com/github/henry4j/-/blob/main/episode_1B.ipynb)

Python Basics: [data structures](https://docs.python.org/3/tutorial/datastructures.html), [deque](https://docs.python.org/3/library/collections.html#collections.deque), [heapq](https://docs.python.org/3/library/heapq.html), [string methods](https://docs.python.org/3/library/stdtypes.html#string-methods), [itertools](https://docs.python.org/3/library/itertools.html#itertools-recipes), [functools](https://docs.python.org/3/library/functools.html), [match-case](https://www.inspiredpython.com/course/pattern-matching/mastering-structural-pattern-matching), [new python features](https://www.nicholashairs.com/posts/major-changes-between-python-versions/), [python tricks](https://sahandsaba.com/14-more-python-features-and-tricks-you-may-not-know.html)

In [ ]:
# @title import before you begin
from functools import lru_cache, partial, reduce
from itertools import accumulate, islice, chain, count, pairwise, product, tee
from collections import namedtuple, Counter, defaultdict, deque
from dataclasses import dataclass
from enum import Enum, IntFlag
from heapq import heappush, heappop, heapify
from typing import Optional, Union
from math import *
from operator import itemgetter
from sys import maxsize as INF  # infinity: 2**63-1.
from sys import setrecursionlimit  # 10_000_000(?)
import bisect
import numpy as np
import operator
import random
import re

class peekable:  # just like more_itertools.peekable
  def __init__(self, iterable, peeked=None):
    self.it, self.peeked = iter(iterable), peeked
  def __iter__(self): return self  # for a for-loop.
  def __next__(self):
    if self.peeked is None:
      return next(self.it)
    peeked, self.peeked = self.peeked, None
    return peeked
  def peek(self, default=None):
    if self.peeked is None:
      try:
        self.peeked = next(self.it)
      except StopIteration:
        return default
    return self.peeked

  @classmethod
  def flatten(cls, iterable):
    def iterate():
      stack = [iter(iterable)]
      while stack:
        it = stack.pop()
        while (e := next(it, None)) is not None:
          if isinstance(e, (list, tuple)):
            stack.append(it)
            it = iter(e)
          else:
            yield e

    return cls(iterate())

Cut = namedtuple("Cut", "start stop", defaults=[0, 0])
Cut.off = classmethod(lambda cls, stop: cls(start=0, stop=stop))
Cut.__abs__ = lambda self: abs(self.stop - self.start)
Cut.__int__ = lambda self: self.stop - self.start
Cut.__call__ = lambda self, seq: seq[self.start:self.stop]

def minima_it(iterable, key=lambda e: e, inverse=False):
  it = tee(iterable, 2)
  key_it = tee(map(key, it[1]), 2)
  min_max = max if inverse else min
  min_max_key = min_max(key_it[1], default=None)
  yield from (e for key, e in zip(key_it[0], it[0]) if key == min_max_key)

def minima(iterable, key=lambda e: e, inverse=False, iterate=False):
  iterable = minima_it(iterable, key, inverse)
  return iterable if iterate else tuple(iterable)

def maxima(iterable, key=lambda e: e):
  it = tee(iterable, 2)
  key_it = tee(map(key, it[1]), 2)
  max_key = max(key_it[1], default=None)
  return tuple(e for key, e in zip(key_it[0], it[0]) if key == max_key)

maxima = partial(minima, inverse=True)
maxima_it = partial(minima_it, inverse=True)
recap = lambda kv, k, v: (kv[0] + k, kv[1] + v)

def pp(obj):
  from collections.abc import Mapping
  _prune = lambda v: (
      dict.fromkeys(v, "…") if isinstance(v, Mapping) else  #
      "…" if isinstance(v, obj.__class__) else v)

  m = len(values := list(_prune(v) for (k, v) in vars(obj).items() if k[0] != "_"))
  n = next((i for i, e in enumerate(reversed(values)) if e is not None), m)
  return f"{obj.__class__.__name__}({repr(values[:m-n])[1:-1]})".replace("'…'", "⋯")

def renumerate(it, stop=None):
  return (L := list(it)).reverse() or zip(count(stop or len(L), -1), L)

def dedupe(iterable):
  seen = set()
  for e in iterable:
    if e not in seen:
      seen.add(e)
      yield e

class DNode:
  def __init__(self, data, prev=None, next_=None):
    self.data, self.prev, self.next = data, prev, next_

def snode_from_values(SNode, *values):
  return reduce(lambda acc, e: SNode(e, acc), reversed(values), None)

@dataclass
class SNode:
  data: int = None
  next: "SNode" = None
  #
  def __iter__(self):
    L = self
    while L:
      yield L
      L = L.next

SNode.from_values = classmethod(snode_from_values)

def btree_from_values(BTree, values, start=0, stop=None):
  if stop is None:
    stop = len(values)
  if stop > start:
    mid = (start+stop-1)//2  # mid becomes start, when stop-start: 2.
    L = btree_from_values(BTree, values, start, mid)
    R = btree_from_values(BTree, values, mid+1, stop)
    return BTree(values[mid], L, R)

class BTree:
  def __init__(self, data=None, left=None, right=None, key=None):
    lr = (c if isinstance(c, (BTree, type(None))) else BTree(c) for c in (left, right))
    self.data, self.left, self.right = data, *lr
    self.key = data if key is None else key
    self._size = sum(bool(c) and c._size for c in (self.left, self.right))+1
    self._x = None

def iterate(tree):  # returns an iterator.
  stack = []
  while tree or stack:
    if tree:
      stack.append(tree)
      tree = tree.left
    else:
      yield (tree := stack.pop())
      tree = tree.right

BTree.__repr__ = pp  # yapf:disable
BTree.from_values = classmethod(btree_from_values)
T = lambda seq: isinstance(seq, list) and list(zip(*seq)) or tuple(zip(*seq))

!! **factorize**, gcd, **primality test** (is prime?), **next_prime** (probabilistic, exact), lcm, etc.

In [ ]:
def factorize(n, d=2):  # n: dividend, d: divisor
  if n==1:
    return tuple()
  elif n % d==0:
    return (d,) + factorize(n // d)
  elif n < d**2:
    return (n,)
  else:
    return factorize(n, 3 if d==2 else d+2)

def gcd(n, d):  # n: dividend, d: divisor.
  while d>0:
    n, d = d, n % d
  return n

def test_primality(n):
  if n in {2, 3}:
    return True
  if n<2 or n % 2==0:
    return False
  return all(n % d for d in range(3, int(n**0.5)+1, 2))  # not divisible: n % d.

def next_prime(n, certainty=5):  # returns when the prob. exceeds 1-0.5 ** certainty.
  if n<4:
    return max(2, n)  # prime p >=2
  if n % 2==0:
    n += 1def factorize(n, d=2):  # n: dividend, d: divisor
  if n==1:
    return tuple()
  elif n % d==0:
    return (d,) + factorize(n // d)
  elif n < d**2:
    return (n,)
  else:
    return factorize(n, 3 if d==2 else d+2)

def gcd(n, d):  # n: dividend, d: divisor.
  while d>0:
    n, d = d, n % d
  return n

def test_primality(n):
  if n in {2, 3}:
    return True
  if n<2 or n % 2==0:
    return False
  return all(n % d for d in range(3, int(n**0.5)+1, 2))  # not divisible: n % d.

def next_prime(n, certainty=5):  # returns when the prob. exceeds 1-0.5 ** certainty.
  if n<4:
    return max(2, n)  # prime p >=2
  if n % 2==0:
    n += 1
  for p in range(n, 2 * n):  # prime p exists where n < p <2n-2 when n >1.
    for _ in range(certainty):
      a = 2 + random.randrange(p-3)  # 2 <= a <= p-2; 2+0 <= a <=2+p-4.
      if 1 != a**(p-1) % p:  # rabin miller's primality test.
        break
    else:
      return p

lcm = lambda a, b: a * b // gcd(a, b)
assert 36 == lcm(2 * 3**2, 2**2 * 3)  # LCM: 2**2 * 3**2.
assert [2, 3, 5, 7, 11] == [e for e in range(12) if test_primality(e)]
assert [2, 2, 2, 3, 5, 5, 7, 7, 11, 11] == [next_prime(n) for n in range(0, 10)]
assert (2, 2, 3) == factorize(12) and (2, 3, 7) == factorize(42)
assert (3, 3, 5) == factorize(45) and (3, 5, 5) == factorize(75)
  for p in range(n, 2 * n):  # prime p exists where n < p <2n-2 when n >1.
    for _ in range(certainty):
      a = 2 + random.randrange(p-3)  # 2 <= a <= p-2; 2+0 <= a <=2+p-4.
      if 1 != a**(p-1) % p:  # rabin miller's primality test.
        break
    else:
      return p

lcm = lambda a, b: a * b // gcd(a, b)
assert 36 == lcm(2 * 3**2, 2**2 * 3)  # LCM: 2**2 * 3**2.
assert [2, 3, 5, 7, 11] == [e for e in range(12) if test_primality(e)]
assert [2, 2, 2, 3, 5, 5, 7, 7, 11, 11] == [next_prime(n) for n in range(0, 10)]
assert (2, 2, 3) == factorize(12) and (2, 3, 7) == factorize(42)
assert (3, 3, 5) == factorize(45) and (3, 5, 5) == factorize(75)

In [ ]:
# @title ##### 1.1 Write a program to determine if a string has all unique characters. What if you cannot use additional data structures?
def uniq(s) -> bool:  # time: O(n), space(n)
  return all(v == 1 for v in histogram(s).values())

def histogram(iterable) -> dict:
  h = {}
  for e in iterable:
    h[e] = h.get(e, 0)+1
  return h

def uniqq(s) -> bool:  # time: o(n^2), space: o(1)
  for i_lhs, lhs in enumerate(s):
    for i_rhs in range(i_lhs+1, len(s)):
      if lhs == s[i_rhs]:
        return False
  return True

assert all([uniq(""), uniq("a"), uniq("ab"), not uniq("aa"), not uniq("aba")])
assert all([uniqq(""), uniqq("a"), uniqq("ab")])
assert not any([uniqq("aa"), uniqq("aba")])

In [ ]:
# @title ###### 1.2 Write a program to test if two strings are permutations of each other.
def anagram(s1, s2):
  if len(s1) != len(s2):
    return False
  h = Counter(s1)
  for e in s2:
    if h.get(e, 0) >0:
      h[e] -= 1
    else:
      return False
  return True

def anagram2(s1, s2):
  signature = lambda s: ''.join(sorted(s))
  return len(s1) == len(s2) and signature(s1) == signature(s2)

def anagram3(s1, s2):
  return Counter(s1) == Counter(s2)

assert anagram('', '') and anagram('a', 'a') and anagram('ab', 'ba')
assert anagram('aab', 'aba') and anagram('aabb', 'abab')
assert not anagram('a', '') and not anagram('', 'a')
assert not anagram('a', 'b') and not anagram('aa', 'ab')
assert anagram2('aab', 'aba') and anagram2('aabb', 'abab')
assert anagram3('aab', 'aba') and anagram3('aabb', 'abab')

In [ ]:
# @title ##### 1.3 Write a program to replace all spaces in a string with %20.
def escape_spaces(s):
  L, m = list(s), len(s)  # m: input length.
  resize(L, n := len(L)+2 * L.count(" "), " ")  # n: output length.
  w = n
  for r in range(m-1, -1, -1):  # i in [m-1, 0].
    buff = "%20" if L[r] == " " else [L[r]]
    w -= len(buff)
    L[w:w + len(buff)] = buff
  return "".join(L)

assert "a%20b%20c%20" == escape_spaces("a b c ")

In [ ]:
# @title ##### 1.4 Write a program to test if a string is a permutation of a palindrome.
palindormic = lambda s: sum(v % 2 for v in Counter(s.lower()).values()) <2
assert all([palindormic(""), palindormic("a"), palindormic("aa"), palindormic("aba")])
assert not any([palindormic("ab"), palindormic("abbc")])

1.5 Write a program to test if two strings are one or zero edits away from each other (insert, delete, or replace).

In [ ]:
def edit(a, b):  # how about this? INS, DEL, SUB = range(1, 4)
  def prog(m, n):
    if m==0 or n==0:
      return (m + n, ((Edit.D,) * m if m>0 else (Edit.I,) * n))
    c = int(a[m-1] != b[n-1])  # edit cost: 1 or 0.
    cases = (
        recap(prog(m-1, n-1), c, ([None, Edit.S][c],)),
        recap(prog(m-1, n-0), 1, (Edit.D,)),
        recap(prog(m-0, n-1), 1, (Edit.I,)),
    )
    return min(cases, key=lambda cost_seq: cost_seq[0])

  return prog(len(a), len(b))

Edit = Enum("Edit", {"I": "Insert", "D": "Delete", "S": "Substitute"})
expected = (3, (Edit.S, None, None, None, Edit.S, None, Edit.I))
assert expected == edit("kitten", "sitting")
assert (3, (Edit.D,) * 3) == edit("cat", "")
assert (3, (Edit.I,) * 3) == edit("", "sit")

In [ ]:
# @title ##### 1.6 Write a method to compress a string using counts of repeated chars, e.g., aabcccccaaa becomes a2b1c5a3.
def compressed(s):
  s2, m, start = [], len(s), 0
  for stop in range(1, m+1):
    if stop == m or s[start] != s[stop]:
      s2.extend([s[start], str(stop - start)])
      start = stop
  return "".join(s2) if len(s2) < m else s

assert "a2b1c5a3" == compressed("aabcccccaaa")
assert "abcc" == compressed("abcc")
assert "abc" == compressed("abc")
assert "" == compressed("")

In [ ]:
# @title ##### 1.7 Given an image represented by an NxN matrix, write a program to rotate the image by 90 degrees; in-place, in O(1) space.
def rotate(g):
  n = len(g)
  for layer in range(n // 2):
    head, tail = layer, n-1 - layer
    for i in range(head, tail):
      top = g[layer][i]
      g[layer][i] = g[n-1 - i][head]  # left to top
      g[n-1 - i][head] = g[tail][n-1 - i]  # bottom to left
      g[tail][n-1 - i] = g[i][tail]  # right to bottom
      g[i][tail] = top  # top to right
  return g

g = [
    [1, 2, 3, 4, 5],
    [6, 7, 8, 9, 10],
    [11, 12, 13, 14, 15],
    [16, 17, 18, 19, 20],
    [21, 22, 23, 24, 25],
]
expected = [
    [21, 16, 11, 6, 1],
    [22, 17, 12, 7, 2],
    [23, 18, 13, 8, 3],
    [24, 19, 14, 9, 4],
    [25, 20, 15, 10, 5],
]
assert expected == rotate(g)

In [ ]:
# @title ##### 1.8 Given an NxN matrix, write a program to set the entire row and column to 0 if an element has a value of 0.
def zero_out(g):
  m, n = len(g), len(g[0])
  rows, columns = set(), set()
  for r in range(m):
    for c in range(n):
      if g[r][c] == 0:
        rows.add(r)
        columns.add(c)
  for r in range(m):
    for c in range(n):
      if r in rows or c in columns:
        g[r][c] = 0
  return g

g = [[1, 2, 3, 4], [5, 6, 7, 8], [9, 0, 1, 2], [3, 4, 5, 6]]
assert [[1, 0, 3, 4], [5, 0, 7, 8], [0, 0, 0, 0], [3, 0, 5, 6]] == zero_out(g)

In [ ]:
# @title ##### 1.9 Given two strings, write a program to test if a string is a rotation of the other using isSubstring method.
is_rotated = lambda s, t: len(s) == len(t) and (s + s).find(t) > -1
assert all([is_rotated("x", "x"), is_rotated("xy", "yx"), is_rotated("xyz", "yzx")])
assert not is_rotated("xyz", "xyx")

In [ ]:
# @title ##### 2.1 Write a program to remove duplicates from an unordered linked list. What if you cannot use additional data structures?
def dedup_o_n_time(L):
  curr, seen = L, {}
  while curr:
    if curr.value in seen:
      pred.next = curr.next
    else:
      seen[curr.value], pred = True, curr
    curr = curr.next
  return L

def dedup_o_1_space(L):
  lhs = L
  while lhs:
    pred = lhs  # predecessor of RHS.
    while pred.next:
      if lhs.value == pred.next.value:
        pred.next = pred.next.next
      else:
        pred = pred.next
    lhs = lhs.next
  return L

L3 = SNode.from_values(1, 2, 3)
assert L3 == dedup_o_n_time(SNode.from_values(1, 1, 2, 3, 3))
assert L3 == dedup_o_n_time(SNode.from_values(1, 2, 1, 2, 3))
assert L3 == dedup_o_1_space(SNode.from_values(1, 1, 2, 3, 3))
assert L3 == dedup_o_1_space(SNode.from_values(1, 2, 1, 2, 3))

In [ ]:
# @title ##### 2.2 Implement an algorithm to find the k-th to last element of a singly linked list.
def last(L: SNode, k=0):
  p, pk = L, None  # pk points to the k-th last node when p reaches the end.
  for _ in range(k):
    if p is None:
      break
    p = p.next
  if p:
    pk = L
    while p.next:
      p, pk = p.next, pk.next
  return pk

L4 = SNode.from_values(*range(4))  # 0, 1, 2, 3
assert [3, 2, 0, None] == [(_ := last(L4, k)) and _.value for k in (0, 1, 3, 4)]

In [ ]:
# @title ##### 2.3 Given access only to a node, implement an algorithm to delete that node in the middle of a singly linked list.

In [ ]:
# @title ##### 2.4 Write a program to partition a linked list around a value of x, such that all nodes less than x come before all nodes greater than or equal to x.
def partition(L, x):
  def push(pool, curr):
    curr.next = pool
    return curr
  def last(curr):
    while curr.next:
      curr = curr.next
    return curr

  lt_x, eq_x, gt_x, = [None], [None], [None]  # 1-element containers.
  curr = L
  while curr:
    next_ = curr.next
    pool = lt_x if curr.data < x else gt_x if curr.data > x else eq_x
    pool[0] = push(pool[0], curr)
    curr = next_
  last(lt_x[0]).next, last(eq_x[0]).next = eq_x[0], gt_x[0]
  return lt_x[0]

L9 = SNode.from_values(9, 3, 8, 2, 5, 6, 1, 7, 4, 5)
assert [4, 1, 2, 3, 5, 5, 7, 6, 8, 9] == [e.data for e in partition(L9, 5)]

2.5. Given two single linked lists where each node has a single digit, write a program that sums them up.

In [ ]:
def sum_integer_strings(L, R):  # Sum two integers in strings.
  L, R = (list(reversed(e)) for e in (L, R))  # becomes a: addend and augend
  outcome = []
  tens = 0
  zero = ord("0")
  for i in range(1 + max(len(L), len(R))):
    ones = tens
    ones += sum(ord(a[i]) - zero for a in (L, R) if i < len(a))
    tens, ones = ones // 10, ones % 10
    outcome.append(chr(zero + ones))
  return "".join(reversed(outcome))

assert "1300" == sum_integer_strings("313", "987")

2.6 Write a program to test if a linked list is palindromic.

2.7 Given two linked lists that interacts by reference (not value), write a program to return the intersecting node.

2.8 Given a linked list, implement an algorithm which returns the node at the beginning of the loop. e.g., INPUT: a -> b -> c -> d -> e -> c, and OUTPUT: c.



3.1 How would you design and implement three stacks on a single array.

In [ ]:
# @title ###### 3.2 Design a stack that has **min** method that returns the minimum in addition to push and pop methods. Push, pop, and min should all operate in O(1) time.
class MinStack:
  def __init__(self):
    self.minimum, self.stack = None, []
  def push(self, e):
    if self.minimum is None or e <= self.minimum:
      self.stack.append(self.minimum)  # saves the previous minimum.
      self.minimum = e
    self.stack.append(e)
    return self
  def pop(self):
    if (e := self.stack.pop()) == self.minimum:
      self.minimum = self.stack.pop()
    return e

S = MinStack().push(2).push(3).push(2).push(1)
assert 1 == S.minimum and 1 == S.pop()
assert 2 == S.minimum and 2 == S.pop()
assert 2 == S.minimum and 3 == S.pop()
assert 2 == S.minimum and 2 == S.pop()
assert S.minimum is None

3.3 Imagine a literal stack of plates. If the stack gets too high, it might topple. Therefore, in real life, we would likely start a new stack when the previous stack exceeds some threshold. Implement a data structure SetOfStacks that mimics this. SetOfStacks should be composed of several stacks and should create a new stack once the previous one exceeds capacity. SetOfStacks.push() and SetOfStacks.pop() should behave identically to a single stack (that is, pop() should return the same values as it would if there were just a single stack). Implement a function popAt(int index) which performs a pop operation on a specific sub-stack.

In [ ]:
# @title ##### 3.5 Design a queue using two stacks.
class Q:  # debug with repr: Q.__repr__ = __repr__
  def __init__(self, inbox=[], outbox=[]):
    self.inbox, self.outbox = inbox, outbox
  def offer(self, E):  # E: element.
    self.inbox.append(E)
    return self
  def poll(self):
    if not self.outbox:
      while self.inbox:
        self.outbox.append(self.inbox.pop())
    return self.outbox.pop() if self.outbox else None

(q := Q()).offer(1).offer(2)
assert 1 == q.poll() and q.offer(3) is q
assert (2, 3, None) == tuple(q.poll() for _ in range(3))

In [ ]:
# @title ##### 3.6 Write a program to sort a stack so the largest elements are at the top. You may use additional stacks to hold items, but you may not copy the elements into any other data structure such as an array. The stack supports the following operations: push, pop, peek, and isEmpty.
def sort(S):  # a = [1, 2, 9, 8, 3, 4]
  L = []
  while S:
    e = S.pop()
    while L and L[-1] > e:
      S.append(L.pop())
    L.append(e)
  S.extend(L)
  return S

assert [1, 2, 3, 4, 5] == sort([1, 2, 5, 3, 4])

3.7 An animal shelter holds only dogs and cats, and operations on a strictly "first in, first out" basis. People must adopt either the oldest (based on the arrival time) of all animals at the shelter, or they can select whether they would prefer a dog or a cat (and will receive the oldest animal of that type). They cannot select which speicific animal they would like. Create the data structures to maintain this system and implement operations such as enqueue, dequeueAny, dequeueDog, and dequeueCat. You may use the LinkedList data structure.

In [ ]:
# @title ##### def optimal_tour(g):
def optimal_tour(g):  # TSP http://www.youtube.com/watch?v=aQB_Y9D5pdw
  def popped(s, i):
    l = list(s)
    l.pop(i)
    return tuple(l)

  # g(v, set) is min([g[v, e] + g(e, set-{e}) for e in set])
  def mapped(v, s, memos={}):
    @lru_cache(maxsize=None)
    def computed(v, s):
      if s:
        return min(((w, g[v][w] + mapped(w, popped(s, i))[1])
                    for i, w in enumerate(s)
                    if g[v][w] != inf),
                   key=lambda e: e[1])
      else:
        return (None, inf) if g[v][0] is inf else (0, g[v][0])

    return computed(v, s)

  return mapped(0, tuple(range(1, len(g))))

g = [
    [0, 10, 15, 20],
    [5,  0,  9, 10],
    [6, 13,  0, 12],
    [8,  8,  9,  0]
]  # yapf:disable
assert (1, 35) == optimal_tour(g)

In [ ]:
# @title ##### Find the shortest paths between all pairs of vertices, [floyd-Warshall](https://en.wikipedia.org/wiki/floyd-Warshall_algorithm).
g = [  #
    [0, inf, -2, inf],
    [4, 0, 3, inf],
    [inf, inf, 0, 2],
     [inf, -1, inf, 0]
]  # yapf: disable
expected = [
    [0, -1, -2, 0],
    [4,  0,  2, 4],
    [5,  1,  0, 2],
    [3, -1,  1, 0]
]  # yapf: disable

def floyd_warshall(graph):  # https://en.wikipedia.org/wiki/floyd-Warshall_algorithm
  d = [r[:] for r in g]  # distance matrix: a deep copy of n x n matrix.
  for k in range(n := len(graph)):  # graph: n x n matrix.
    for i in range(n):
      for j in range(n):
        d[i][j] = min(d[i][j], d[i][k] + d[k][j])
  return d

assert expected == floyd_warshall(g)

!! **vertex-color, edge-color** Given a list of 1:1 meetings, find the minimum number of time slots for scheduling them all. No one can be in more than one meeting a time.

In [ ]:
def color_vertex(g):  # yield a tuple (chromatic index, color sequence).
  def backtrack(color_seq):
    if len(color_seq) == len(g):
      yield tuple(color_seq)
    else:
      for color in expand_out(color_seq):  # color c may be an old color, or a new one.
        color_seq.append(color)
        yield from backtrack(color_seq)
        color_seq.pop()
  def within_bounds_n_limits(color_seq, c):  # is color c within constraints?
    v = len(color_seq)
    return all([not g[v0][v] or c0 != c for v0, c0 in enumerate(color_seq)])
  def expand_out(color_seq):  #
    assert len(color_seq) < len(g)
    yield from chain(
        (c for c in set(color_seq) if within_bounds_n_limits(color_seq, c)),
        [chromatic(color_seq)])

  chromatic = lambda color_seq: max(color_seq)+1
  yield from ((chromatic(e), e) for e in backtrack([0]))

# a0 - b1
# |    |
# c2 - d3 - e4
G = [
    [0, 1, 1, 0, 0],
    [1, 0, 0, 1, 0],
    [1, 0, 0, 1, 0],
    [0, 1, 1, 0, 1],
    [0, 0, 0, 1, 0],
]
# (3, (0, 1, 1, ...)): color: chromatic index 3, color sequence: (0, 1, 1, ...).
# assert [(2, (0, 1, 1, 0, 1))] ==
min(color_vertex(G), key=lambda e: e[0])

!! **iterate, riterate a binary search tree** using a stack; iterative inorder traversal without recursion.

In [ ]:
def riterate(tree):  # returns a reverse iterator.
  stack = []
  while tree or stack:
    if tree:
      stack.append(tree)
      tree = tree.right
    else:
      yield (tree := stack.pop())
      tree = tree.left

L = BTree(3, BTree(2, BTree(1, BTree(0))))
R = BTree(8, BTree(5, None, BTree(7, BTree(6))), BTree(9))
t10 = BTree(4, L, R)
assert list(range(10)) == list(tree.key for tree in iterate(t10))
assert [9, 8] == list(e.key for e in islice(riterate(t10), 2))
assert [8, 7] == list(e.key for e in islice(riterate(t10), 1, 3))
assert [8, 6, 4] == list(e.key for e in islice(riterate(t10), 1, 6, 2))

!! **acyclic or cyclic** Find a cycle if one exists? Detect a deadlocked if one exists?

* undirected: it is a tree if the undirected graph is connected and has n - 1 edges for n vertices.
* directed: a directed acyclic graph has no back edges; a cyclic graph has back edges.

In [ ]:
def find_cycles(G, directed=False):  # G: graph either directed, or undirected.
  def dfs(v, entered, exited, directed, parent):
    if v not in entered:
      entered.add(v)
      for w in G[v]:
        if w not in entered:
          yield from dfs(w, entered, exited, directed, v)
        elif directed and w not in exited:  # a back edge, unless w is exited.
          yield (v, w)
        elif not directed and parent != w:  # a back edge, unless w is a parent.
          yield (v, w)
      exited.add(v)

  entered, exited = set(), set()
  yield from (cycle  #
              for v in G  #
              for cycle in dfs(v, entered, exited, directed, parent=None))

"""graph:
A0 →  C2 → E4
↓  ↗  ↓   ↗
B1 .  D3"""
a, b, c, d, e = range(5)
g = {
  a: {b, c},
  b: {c},
  c: {d, e},
  d: {e},
  e: {}
}  # yapf:disable
assert not next(find_cycles(g, True), None)

g[a] = {b}
g[c] = {a, d, e}  # this change implants a cycle.
assert [(2, 0)] == list(find_cycles(g, True))

# undirected graph: A0 - B1 - C2
a, b, c = range(3)
g2 = {a: {b}, b: {a, c}, c: {b}}
assert not next(find_cycles(g2, False), None)

# undirected graph: A0 - B1 - C2 - A0
g2[a].add(c)
g2[c].add(a)
assert [(2, 0), (0, 2)] == list(find_cycles(g2, False))

!! **topological sort, reachable, connected components**

4.7 Given a set of projects and their dependencies. Find a build order that ensures each project is built only after its dependencies are built.  
4.1 Given a directed graph, design an algorithm to find whether there is a route between two nodes.

In [ ]:
# E = Edge = namedtuple("Edge", "w v weight", defaults=[-1, 0])  # v to w edge.
E = Edge = namedtuple("Edge", "w v", defaults=[-1])  # v to w edge.
E.replace = E._replace
OP = OpCode = IntFlag("OpCode", "ENTER CROSS EXIT")

def dfs(graph, initial_edge, entered=None):  # initial_edge, also called start_edge.
  if entered is None:
    entered = set()
  if (v0 := initial_edge.w) not in entered:
    entered.add(v0)
    yield OP.ENTER, initial_edge
    for edge in graph[v0] or []:
      yield OP.CROSS, (edge := edge._replace(v=v0))
      yield from dfs(graph, edge, entered)
    yield OP.EXIT, initial_edge

def topological_sort(graph):
  entered = set()
  for v, _ in enumerate(graph):
    if v not in entered:
      yield from (
          edge.w for op, edge in dfs(graph, Edge(v, -1), entered) if op == OP.EXIT)

def can_reach(graph, source, sink) -> bool:
  return any(edge.w == sink for op, edge in dfs(graph, Edge(source)) if op == OP.CROSS)

# graph: .  .  D3 ⇾ H7
#  .  .  .  .  ↑
#   ┌───────── B1 ⇾ F5
#   ↓  .  .  . ↑  .  ↑
#   J9 ⇽ E4 ⇽ A0 ⇾ C2 ⇾ I8
#   ↓
#   G6
graph = [  #
    [Edge(1), Edge(2), Edge(4), Edge(6)],  # 1, 2, 4, and 6 depend on 0.
    [Edge(3), Edge(5), Edge(9)],  # 3, 5, and 9 depend on 1.
    [Edge(5), Edge(8)],  # 5, 8 depend on 2.
    [Edge(7)],  # 7 depend on 3.
    [Edge(9)],  # 9 depend on 4.
    *([[]] * 5),
]
stack = [7, 3, 5, 9, 1, 8, 2, 4, 6, 0]  # task 0 be completed before task 6.
assert stack == list(topological_sort(graph))
assert all(can_reach(graph, source, 9) for source in (0, 1, 4))
assert not any(can_reach(graph, source, 0) for source in range(4))

In [ ]:
# @title ##### 4.2 Given a sorted (increasing order) array, design an algorithm to create a binary search tree with the minimal height.
# tree:  2
#      /  \
#     1    3
#           4
t4 = BTree.from_values((1, 2, 3, 4))
assert "BTree(2, BTree(1), BTree(3, None, BTree(4)))" == repr(t4)

In [ ]:
# @title ##### 4.3 Given a binary tree, design an algorithm to creates a linked list of all the nodes at each depth, e.g., if you have a tree with depth D, you will have D linked lists.
def to_d_linked_lists(tree):  # tree of depth d.
  LL = []  # output: a list of lists.
  q = [tree]
  while q:
    p = []
    for i, e in enumerate(q):
      for c in (e.left, e.right):
        if c:
          p.append(c)
      e.left = None if i == 0 else q[i - 1]
      e.right = q[i + 1] if i < len(q) else None
    LL.append(q[0])
    q = p
  return LL

In [ ]:
# @title ##### 4.4 Implement a function to check if a binary tree is balanced. For the purposes of this question, a balanced tree is defined to be a tree such that the heights of two subtrees of any node never differ by more than one.
# tree: a
# . .  /   \
# .   b  .  f
#   c  e   g
#  d
c = BTree("c", BTree("d"))
e = BTree("e")
b = BTree("b", c, e)
a = BTree("a", b, BTree("f"))

def is_balanced(tree):  # returns (balanced, height)
  if tree is None:
    return (True, -1)
  L, R = is_balanced(tree.left), is_balanced(tree.right)
  b = L[0] and R[0] and abs(L[1] - R[1]) <2
  h = 1 + max(L[1], R[1])
  return (b, h)

assert not is_balanced(a)[0]
a.right.left = BTree("g")
assert (True, 3) == is_balanced(a)  # balanced, height: 3.

In [ ]:
# @title ##### 4.5 Implement a function to check if a binary tree is a binary search tree.
def iterate_(tree):
  if tree:
    yield from chain(iterate_(tree.left), (tree,), iterate_(tree.right))

def is_ordered(tree):
  if pred := next(it := iterate_(tree), None):
    for e in it:
      if e.data < pred.data:
        return False
      else:
        pred = e
  return True

def ordered(tree):  # returns (ordered, min, max)
  if tree is None:
    return (True, None, None)
  L, R = ordered(tree.left), ordered(tree.right)
  O = (L[0] and R[0]
       and (L[2] is None or L[2] <= tree.data)
       and (R[1] is None or R[1] >= tree.data))  # yapf:disable
  min_ = tree.data if L[1] is None else L[1]
  max_ = tree.data if R[2] is None else R[2]
  return (O, min_, max_)

b7 = BTree.from_values([1, 2, 3, 4, 5, 6, 7])
assert is_ordered(b7)
assert ordered(b7)[0]
assert not ordered(BTree.from_values((1, 2, 3, 4, 0, 6, 7)))[0]

In [ ]:
# @title ##### 4.6 Design an algorithm to find the next node (i.e., successor in-order) of a node in a binary search tree. You may assume that each node has a link to its parents.
# tree: .  z
#  .  .  u
#  .  .  . v
#  .  .  .   y
#  .  .  . x
#  .  .  w
w = BTree("w")
x = BTree("x", w)
y = BTree("y", x)
v = BTree("v", None, y)
u = BTree("u", None, v)
z = BTree("z", u)

def set_parent(tree):  # set value fields on the children/descendants.
  tree._parent = None
  stack = []
  while tree or stack:
    if tree:
      stack.append(tree)
      if tree.left:
        tree.left._parent = tree
      tree = tree.left
    else:
      tree = stack.pop()
      if tree.right:
        tree.right._parent = tree
      tree = tree.right

def succ(node):
  if node is None:
    raise RuntimeError("'node' must be non-null.")
  if node.right:
    node = node.right
    while node.left:
      node = node.left
    return node
  else:
    while node._parent and node == node._parent.right:
      node = node._parent
    return node._parent

set_parent(z)
assert (v, w, x, y, z, None) == tuple(succ(tree) for tree in (u, v, w, x, y, z))

In [ ]:
# @title ##### 4.8 Design an algorithm to find the first common ancestor of the nodes in a binary tree. Avoid storing additional nodes in a data structure. Note: this is not necessarily a binary search tree.
def is_subtree(tree, subtree):
  return (
    tree == subtree
    or tree
    and (is_subtree(tree.left, subtree) or is_subtree(tree.right, subtree))
  )

def lowest_common_ancestor(tree, p, q):
  if is_subtree(tree, p) and is_subtree(tree, q):
    while tree:
      if tree is p or tree is q:
        return tree
      p_on_left = is_subtree(tree.left, p)
      q_on_left = is_subtree(tree.left, q)
      if p_on_left != q_on_left:
        return tree
      tree = tree.left if p_on_left else tree.right

def lowest_common_ancestor2(tree, p, q):  # returns (count, LCA)
  if tree is None:
    return (None, 0)
  count = (p, q).count(tree)
  if count == 2:
    return (tree, 2)
  l = lowest_common_ancestor2(tree.left, p, q)
  if l[1] == 2:
    return l
  r = lowest_common_ancestor2(tree.right, p, q)
  if r[1] == 2:
    return r
  count += l[1] + r[1]
  return (tree if count == 2 else None, count)

# tree:.   a
#  .  .  /   \
#  .   b  .   f
#  .  c  e  g
#   d
d = BTree("d")
c = BTree("c", d)
e = BTree("e")
b = BTree("b", c, e)
a = BTree("a", b, BTree("f", BTree("g")))
assert is_subtree(a, a)
assert is_subtree(a, a.left) and is_subtree(a, a.left.right)
assert is_subtree(a, a.right) and is_subtree(a, a.right.left)
assert b == lowest_common_ancestor(a, d, e)
assert b == lowest_common_ancestor2(a, d, e)[0]

In [ ]:
# @title ##### 4.9 Given a binary search tree that was reconstructed from an array of integers, find all the possible arrays that could have led to this binary search tree.
def weave(L, R):
  @lru_cache(maxsize=None)
  def prog(m, n):
    if m==0 or n==0:
      yield L[:m] + R[:n]
    else:
      yield from chain(
          (e + [L[m-1]] for e in prog(m-1, n)),
          (e + [R[n-1]] for e in prog(m, n-1)),
      )

  yield from prog((m := len(L)), (n := len(R)))

def destruct(tree: BTree):
  if tree is None:
    yield []
  else:
    yield from ([tree.data] + e
                for L in destruct(tree.left)
                for R in destruct(tree.right)
                for e in weave(L, R))

T5 = BTree(4, BTree(1, None, BTree(3, BTree(2))), BTree(5))
expected = ([4, 5, 1, 3, 2], [4, 1, 5, 3, 2], [4, 1, 3, 5, 2], [4, 1, 3, 2, 5])
assert expected == tuple(destruct(T5))

!! **serialize, deserialize, preorder, bfs**

4.10 You have two very large binary trees: T1 with millions of nodes, and T2 with hundreds of nodes. Design an algorithm to decide if T2 is a subtree of T1. A tree T2 is a subtree of T1 if there exists a node in T1 such that the subtree of n is identical to T2. i.e., if you cut off the tree at node n, the two trees would be identical.

In [ ]:
def search_btree(tree, query, fill_value=BTree(-1)):  # query: a subtree.
  needle = deque(e.data for e in preorder(query, fill_value))
  m = len(needle)
  it = tee(preorder(tree, fill_value))
  haystack = deque((e.data for e in islice(it[0], m-1)), maxlen=m)
  for tree_in, tree_out in zip(it[0], it[1]):  # tree_out: m-1 steps ahead.
    haystack.append(tree_in.data)
    if haystack == needle:
      yield tree_out  # note: it[1] is m-1 steps behind it[0].

def preorder_quick(tree, fill_tree=None):  # fill_value, e.g. BTree(-1)
  stack = [tree]
  while stack:
    tree = stack.pop()
    if tree:
      yield tree
      stack.extend((tree.right, tree.left))
    elif fill_tree:
      yield fill_tree

def preorder(tree, fill_tree=None):
  if tree:
    yield tree
    yield from preorder(tree.left, fill_tree)
    yield from preorder(tree.right, fill_tree)
  elif fill_tree:
    yield fill_tree

def btree_from_preorder_data(iterable, fill_data=None):  # fill_data, e.g., -1
  def proc():
    if (data := next(it, fill_data)) != fill_data:
      return BTree(data, proc(), proc())

  it = iter(iterable)
  return proc()

# tree:  4
#      /   \
#     3     8
#    /     /  \
#   2     5    9
#  1       7
# 0       6
# n.b.: t4 has two instances of t5 subtree.
t5 = BTree(5, None, BTree(7, BTree(6)))
t3 = BTree(3, BTree(2, BTree(1, BTree(0, t5))))
t8 = BTree(8, t5, BTree(9))
t4 = BTree(4, t3, t8)
assert (5, 5) == tuple(e.data for e in search_btree(t4, t5))
assert (8,) == tuple(e.data for e in search_btree(t4, t8))
assert (4,) == tuple(e.data for e in search_btree(t4, t4))

fill_value = BTree(-1)
picked_t8 = (e.data for e in preorder(t8, fill_value))
t = btree_from_preorder_data(picked_t8, fill_value.data)
assert [8, 5, 7, 6, 9] == [e.data for e in preorder(t)]
assert [5, 6, 7, 8, 9] == [e.data for e in iterate(t)]

In [ ]:
def btree_from_bfs_data(iterable, fill_data=-1):
  q = deque([tree_out := BTree(next(it := iter(iterable), fill_data))])
  for i, d in enumerate(it, 1):
    if (tree := BTree(d) if d != fill_data else None):
      q.append(tree)
    if i%2:
      q[0].left = tree
    else:
      q[0].right = tree
      q.popleft()
  return tree_out

def bfs(tree_in, fill_tree=BTree(-1)):
  def proc():
    q = deque([(0, tree_in)])
    while q:
      d, tree = q.popleft()
      if tree:
        yield (d, tree)
        q.extend([(d+1, tree.left), (d+1, tree.right)])
      elif fill_tree:
        yield (d, fill_tree)
  it2 = tee(proc())
  max_d = max(d for d, t in it2[1]) if fill_tree else -1
  yield from (t for d, t in it2[0] if d != max_d)

pickled = (1, 2, 3, -1, -1, 4, 5, -1, -1, -1, 6)
b6 = BTree(1, BTree(2), BTree(3, BTree(4), BTree(5, None, BTree(6))))
#      1
#   2   3
#      / \
#     4   5
#          6
pickle = lambda tree: (e.data for e in bfs(tree, BTree(-1)))
assert pickled == tuple(pickle(btree_from_bfs_data(pickle(b6), -1)))

In [ ]:
class Tree:
  def __init__(self, data):
    self.data, self.children = data, []

def preorder(tree, fill_tree=Tree("#")):
  if tree:
    yield tree
    if tree.children:
      yield from chain.from_iterable(preorder(c, fill_tree) for c in tree.children)
    if fill_tree:
      yield fill_tree

def tree_from_preorder_data(iterable, fill_data="#"):  # fill_data, e.g., -1
  def proc():
    while (data := next(it, fill_data)) != fill_data:
      tree = Tree(data)
      while (child := proc()):
        tree.children.append(child)
      return tree

  it = iter(iterable)
  return proc()

# tree
#        7
#     /     \
#   5         9
# 2 3 6
t7 = Tree(7)
t5 = Tree(5)
t5.children.extend([Tree(2), Tree(3), Tree(6)])
t7.children.extend([t5, Tree(9)])

pickled = "7,5,2,#,3,#,6,#,#,9,#,#"
assert pickled == ",".join(str(e.data) for e in preorder(t7))
t7_clone = tree_from_preorder_data(pickled.split(","))
assert pickled == ",".join(str(e.data) for e in preorder(t7_clone))

In [ ]:
def diameter(tree):  # return (diameter, height).
  if tree is None:
    return (0, 0)
  L = diameter(tree.left)
  R = diameter(tree.right)
  dia = max(L[0], R[0], 1 + L[1] + R[1])
  return (dia, height := 1 + max(L[1], R[1]))

def btree_from_two_orders(preorder, inorder):
  if preorder:
    value = preorder[0]
    m = inorder.index(value)
    return BTree(value, btree_from_two_orders(preorder[1:m+1], inorder[:m]),
                 btree_from_two_orders(preorder[m+1:], inorder[m+1:]))

# tree input:   a
#             b
#          c    f
#           d     g
#            e
tree = btree_from_two_orders("abcdefg", "cdebfga")
assert (6, 5) == diameter(tree)

In [ ]:
def isomorphic(tree, tester):  # for value equality, not reference equality.
  return (
    tree is None and tester is None
    or tree and tester
       and tree.data == tester.data
       and isomorphic(tree.left, tester.left)
       and isomorphic(tree.right, tester.right)
  )  # yapf:disable

def contains(tree, subtree, match_fn=isomorphic):
  return (
    subtree is None  # always true, if subtree is None.
    or match_fn(tree, subtree)
    or tree
       and (contains(tree.left, subtree, match_fn)
            or contains(tree.right, subtree, match_fn))
  )  # yapf:disable
Fse
def starts_with(tree, tester):
  return (
    tester is None
    or tree
    and tree.data == tester.data
    and starts_with(tree.left, tester.left)
    and starts_with(tree.right, tester.right)
  )  # yapf:disable

!! **sequence sum** Given a binary tree of integers, find paths that sums up to a value. A path can start at any subtrees. e.g., paths (3, -1, 2), and (-1, 3, -1, 3) that sum up to 4 given this tree.
```
tree: -1
       \
        3
       /  \
     -1    2
     / \
    2   3
```

In [ ]:
def find_seq_sum(tree, seq_sum, cumsum_seq=[0], cumsum_indices=None):
  if tree:  # diff works like np.diff, that reverses np.cumsum.
    diff = lambda cumsum_seq: tuple(q - p for p, q in pairwise(cumsum_seq))
    cumsum_seq.append(cumsum := cumsum_seq[-1] + tree.data)
    cumsum_indices = cumsum_indices or defaultdict(list, {seq_sum: [0]})
    cumsum_indices[cumsum + seq_sum].append(len(cumsum_seq)-1)
    if cumsum_indices[cumsum]:
      yield from (diff(cumsum_seq[i:]) for i in cumsum_indices[cumsum])
    yield from chain(
        find_seq_sum(tree.left, seq_sum, cumsum_seq, cumsum_indices),
        find_seq_sum(tree.right, seq_sum, cumsum_seq, cumsum_indices))
    cumsum_seq.pop()
    cumsum_indices[cumsum + seq_sum].pop()

tree = BTree(-1, None, BTree(3, BTree(-1, BTree(2), BTree(3)), BTree(2)))
assert ((3, -1, 2), (-1, 3, -1, 3), (-1, 3, 2)) == tuple(find_seq_sum(tree, 4))

In [ ]:
# @title ##### 5.2 Given a double-precision floating-point number between 0 and 1, output its binary representation. If the exact binary representation requires more than 32 bits, print an error message.
def float_to_str(f: float) -> str:
  if f <= 0 or f >= 1:
    return "ERROR"
  L = list(".")
  while f > 0:
    if len(L) > 31:
      return "ERROR"
    f = round(2 * f, 6)
    if f >= 1:
      L.append("1")
      f -= 1
    else:
      L.append("0")
  return "".join(L)

assert ".011" == float_to_str(0.375)
assert ".101" == float_to_str(0.625)

In [ ]:
# @title ##### 5.3 Given an integer, write a program to find the length of the longest sequence of 1s after flipping exactly one 0 to 1.

!! **next k bit_length sequence (successor), nCk (n choose k)** 5.4 Get the predecessor and successor integers with the same number of bits (1s).

In [ ]:
def successor(n: int) -> int:  # count 0s, and 1s to process.
  m = n.bit_length()
  count0 = next((c0 for c0 in range(m) if n & (1 << c0)), 0)  # c0: no. 0s.
  count1 = next((c1 for c1 in range(count0, m) if n & (1 << c1)==0), m) - count0
  return n + (1 << count0) + (1 << count1-1)-1

def predecessor(n: int) -> int:  # count 1s and 0s to process.
  m = n.bit_length()
  count1 = next((c1 for c1 in range(m) if n & (1 << c1)==0), m)  # c1: no. 1s.
  count0 = next((c0 for c0 in range(count1, m) if n & (1 << c0)), 0) - count1
  return n - (1 << count1) - ((1 << count0-1)-1)

pred, succ = 256+16+8+4, 256+32+2+1
assert succ == successor(pred) and pred == predecessor(succ)

5.5 What does this code do? (n > 0 and n & (m-1) == 0)

5.6 Write a program to get the Hamming distance between two integers -- the number of positions at which the corresponding bits are different. `(lhs ^ rhs).bit_count()`.


5.7 Write a program to swap odd and even bits in an integer with as few instructions as possible.  
`swap_bits = lambda n: (n & 0xAAAAAAAA) >> 1 | (n & 0x55555555) << 1`

In [ ]:
# @title ##### 5.8 Design an algorithm to draw a horizontal line on a monochrome screen, where the screen is represented as a byte array with each byte storing 8 pixels. The function signature looks like: draw_line(screen: bytes[], width, x1, x2, y: int)

In [ ]:
# @title ##### 8.1 Given a staircase with n steps, write a program to count the number of possible ways to climb it, when one can hop either 1, 2, or 3 steps at a time.
@lru_cache(maxsize=None)
def climb(n):
  if n in (1, 2):
    return n
  if n==3:
    return 4
  return climb(n-1) + climb(n-2) + climb(n-3)

assert 13 == climb(5)
assert 24 == climb(6)

8.2 Given n x m grid, write a program to route a robot from (0, 0) to (n-1, m-1). How many possible ways are there, when the robot can move in two directions: right, and down. What if there are some spots of off-limits?

In [ ]:
def backtrack(candidate):  # backtracks from a partial route candidate.
  def within_bounds_n_limits(r, c):  # bounds: 0s. Q: should you avoid cycles?
    return 0 <= r < m and 0 <= c and c < n and maze[r][c]==1
  def expand_out(candidate):
    r0, c0 = candidate[-1]
    for r, c in ((r0+1, c0), (r0, c0+1)):
      if within_bounds_n_limits(r, c):
        yield (r, c)

  if candidate[-1] == (m-1, n-1):
    yield tuple(candidate)  # reduces to a complete route candidate.
  else:
    for e in expand_out(candidate):
      candidate.append(e)
      yield from backtrack(candidate)
      candidate.pop()

maze = [  # maze: m x n grid.
    [1, 1, 1, 1, 1, 0],
    [1, 0, 0, 0, 1, 0],
    [1, 1, 1, 1, 1, 0],
    [1, 0, 0, 0, 1, 1],
    [1, 1, 0, 0, 1, 0],
    [1, 1, 0, 1, 1, 1],
]
m, n = len(maze), len(maze[0])

expected = (
  ((0, 0), (1, 0), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (3, 4), (4, 4), (5, 4), (5, 5)),
  ((0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4), (5, 4), (5, 5)),
)  # yapf: disable
assert expected == tuple(islice(backtrack([(0, 0)]), 0, 2))

8.3 Given an array of sorted integers, write a method to find a magic index where A[i] = i. What if integers are not distinct?

In [ ]:
def magic(L, lo=0, hi=None):
  if hi is None:
    hi = len(L)
  if hi - lo>0:
    mid = (lo+hi-1) // 2
    if mid == L[mid]:
      return mid
    return (magic(L, lo, min(mid, L[mid]+1)) or magic(L, max(mid+1, L[mid]), hi))

deq = [  #
    [1, 3, 3, 5, 5, 5, 7, 9, 9],
    [1, 3, 3, 5, 6, 6, 7, 7, 9],
    [1, 1, 3, 4, 6, 6, 7, 9, 9],
]
assert [5, 7, 1] == [magic(q) for q in deq]

!! **onesided bisect left, bisect range, bisect_left, bisect_right** Find the exact point of transition within an array that contains a run of 0's and an unbounded run of 1's.

In [ ]:
def one_sided_bisect(L):  # one-sided binary search; algorithm design manual 4.9.2.
  if L[1]==1:
    return 1
  for n in range(1, 32):
    if L[2**n]==1:
      return bisect_left(L, 1, lo=2**(n-1)+1, hi=2**n+1)

def bisect_left(L, x, lo=0, hi=None):
  if hi is None:
    hi = len(L)
  while lo < hi:
    mid = (lo+hi) // 2
    if x > L[mid]:
      lo = mid+1
    else:
      hi = mid
  return lo

def bisect_right(L, x, lo=0, hi=None):
  if hi is None:
    hi = len(L)
  while lo < hi:
    mid = (lo+hi) // 2
    if x >= L[mid]:
      lo = mid+1
    else:
      hi = mid
  return lo

def bisect_range(L, x, lo=0, hi=None):
  if hi is None:
    hi = len(L)
  args = [L, x, lo, hi]
  return (bisect_left(*args), bisect_right(*args))

L = [1, 1, 5, 5, 5, 5, 5, 7, 7]
assert [(2, 7), (0, 2), (7, 9)] == [bisect_range(L, x) for x in (5, 1, 7)]
assert [(0, 0), (9, 9)] == [bisect_range(L, x) for x in (0, 8)]
test_cases = ((0, 1), (0, 0, 1), (0, 0, 0, 1, 1), (0, 0, 0, 0, 1))
assert [1, 2, 3, 4] == [one_sided_bisect(seq) for seq in test_cases]

!! **permutate next, permutations, combinations (all subsets)** ilen(permutations(range(5), 2)) == 20 and ilen(combinations(range(5), 2)) == 10

* a k-combination of a set S is a subset of k distinct elements of S, and the # of k-combinations is equals to the binomial coefficient, n! / (k! * (n-k)!).
* a k-permutation of a set S is an ordered sequence of k distinct elements of S, and the # of k-permutation of n objects is denoted variously nPk, Pn,k, and P(n,k), and its value is given by n! / (n-k)!.

---

8.4 Write a program to generate all subsets of a set.  
8.7 and 8.8 Write a program to generate all permutations of a string of unique and non-unique characters.

In [ ]:
# Find the longest rising tail sequence (from the end), swap with L[j] greater L[i], and reverse.
# 2, 4, 3, 1 -> 3, 1, 2, 4
# i           : i where seq[i] smaller than seq[i+i]
#       j     : j where seq[j] larger  than seq[i]
# 3     2
# 3  1  2  4
def permutate_next(seq):  # returns the next lexicographical permutation, or None.
  n = len(seq)
  if (i := next((i for i in range(n-2, -1, -1) if seq[i] < seq[i+1]), -1)) > -1:
    j = next(j for j in range(n-1, i, -1) if seq[j] > seq[i])
    seq[i], seq[j] = seq[j], seq[i]
    seq[i+1:] = seq[i+1:][::-1]  # reverses the descending sequence.
    return seq

def all_subsets(L, stop=None):  # nCk combinations, where k: [0, n].
  if (stop := len(L) if stop is None else stop)==0:
    yield ()
  else:
    for subseq in all_subsets(L, stop-1):
      yield from (subseq + (L[stop-1],), subseq)

def permutate(L, i=0):  # permutates L at index i.
  if i == len(L):
    yield L
  else:
    distinct_value_indices = {L[j]: j for j in range(i, len(L))}
    for _, j in distinct_value_indices.items():
      L[i], L[j] = L[j], L[i]
      yield from permutate(L, i+1)
      L[i], L[j] = L[j], L[i]

assert [2, 1, 3, 4] == permutate_next([1, 4, 3, 2])
assert not permutate_next([4, 3, 2, 1])
assert ["aab", "aba", "baa"] == list("".join(e) for e in permutate(list("aab")))
expected = "abc", "ab", "ac", "a", "bc", "b", "c", ""
assert expected == tuple(map("".join, all_subsets("abc")))

def combinations(iterable, k):  # k combinations, C(n, k), often read as "n choose k".
  pool = tuple(iterable)
  n = len(pool)
  if k > n:
    return
  indices = list(range(k))
  yield tuple(pool[i] for i in indices)
  while True:
    for i in reversed(range(k)):  # Q: any not maxed out.
      if indices[i] != i + n - k:
        break
    else:
      return  # Looks all maxed out.
    indices[i:k] = range(indices[i]+1, indices[i] + k - i+1)
    yield tuple(pool[i] for i in indices)

assert "AB AC AD BC BD CD".split(" ") == list(map("".join, combinations("ABCD", 2)))

def permutations(iterable, r=None):
  pool = tuple(iterable)
  n = len(pool)
  r = n if r is None else r
  if r > n:
    return

  indices = list(range(n))
  cycles = list(range(n, n - r, -1))
  yield tuple(pool[i] for i in indices[:r])

  while n:
    for i in reversed(range(r)):
      cycles[i] -= 1
      if cycles[i]==0:
        indices[i:] = indices[i+1:] + indices[i:i+1]
        cycles[i] = n - i
      else:
        j = cycles[i]
        indices[i], indices[-j] = indices[-j], indices[i]
        yield tuple(pool[i] for i in indices[:r])
        break
    else:
      return

assert ("AB AC AD BA BC BD CA CB CD DA DB DC".split(" ")  #
        == list(map("".join, permutations("ABCD", 2))))

In [ ]:
# @title ##### 8.9 Write a program to generate all possible, valid combinations of n-pairs of parenthesis, e.g., INPUT: 3, OUTPUT: ((())), (()()), (())(), ()(()), ()()().
def combine_parens(o, c):  # Combines parens with open and close balanced.
  if c==0:
    yield ""
  elif o==0:
    yield ")" * c
  elif c == o:
    yield from ("(" + e for e in combine_parens(o-1, c))
  else:
    yield from ("(" + e for e in combine_parens(o-1, c))
    yield from (")" + e for e in combine_parens(o, c-1))

assert {'((()))', '(()())', '(())()', '()(())', '()()()'} == set(combine_parens(3, 3))

!! **flood-fill** 8.10 Write a flood-fill method to fill in a new color until the color changes from the original color; given a point and a new color.


In [ ]:
def flood_fill(G, r, c, new):  # G: graph.
  m, n, old = len(G), len(G[0]), G[r][c]  # m rows x n columns.
  stack = [(r, c)]
  while stack:
    r, c = stack.pop()
    if 0 <= r < m and 0 <= c < n and G[r][c] == old:
      G[r][c] = new
      for dr, dc in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        stack.append((r + dr, c + dc))
  return G

def flood_fill_quick(graph, r, c, new):  # graph g: m x n pixel image
  m, n, G, old = len(graph), len(graph[0]), graph, graph[r][c]  # m rows x n columns.
  q = deque([(r, c)])
  while q:
    r, c = q.popleft()
    if G[r][c] == old:
      west = next((w for w in reversed(range(c)) if G[r][w] != old), -1)+1
      east = next((e for e in range(c+1, n) if G[r][e] != old), n)-1
      for c in range(west, east+1):
        G[r][c] = new
        if r>0 and G[r-1][c] == old:
          q.append((r-1, c))
        if r < m-1 and G[r+1][c] == old:
          q.append((r+1, c))
  return G

# yapf:disable
graph = lambda: [
    [1, 1, 1, 1],
    [1, 0, 0, 1],
    [0, 1, 0, 1],
    [1, 1, 1, 1]
]
assert [[1, 1, 1, 1], [1, 2, 2, 1], [0, 1, 2, 1], [1, 1, 1, 1]] == flood_fill(graph(), 1, 1, 2)
assert [[1, 1, 1, 1], [1, 3, 3, 1], [0, 1, 3, 1], [1, 1, 1, 1]] == flood_fill(graph(), 2, 2, 3)
assert [[4, 4, 4, 4], [4, 0, 0, 4], [0, 4, 0, 4], [4, 4, 4, 4]] == flood_fill(graph(), 2, 1, 4)
assert [[1, 1, 1, 1], [1, 3, 3, 1], [0, 1, 3, 1], [1, 1, 1, 1]] == flood_fill_quick(graph(), 2, 2, 3)
assert [[4, 4, 4, 4], [4, 0, 0, 4], [0, 4, 0, 4], [4, 4, 4, 4]] == flood_fill_quick(graph(), 2, 1, 4)

8.11 Given infinite # of coins (25, 10, 5, and 1 cents), write a method to count the number of ways to represent n cents.

In [ ]:
def make_amount(k, denominations, i=0):  # Processes a denom. at index i.
  d_i = denominations[i]
  return sum(make_amount(k - m*d_i, denominations, i+1)
             for m in range(k//d_i+1)) if d_i>1 else 1

def make_changes(k, denoms):  # Make the changes k with least coins.
  def prog(k):  # needs \@lru_cache(maxsize=None)
    cases = ((d,) + prog(k - d) for d in denoms if k >= d)
    return min(cases, key=lambda v: len(v)) if k>0 else ()

  return prog(k)

assert 1 == make_amount(4, [25, 10, 5, 1])
assert 2 == make_amount(5, [25, 10, 5, 1])
assert 4 == make_amount(10, [25, 10, 5, 1])  # 4 ways to make 10 cents
assert 9 == make_amount(20, [25, 10, 5, 1])  # 13 ways to make 25 cents.
assert (5, 5) == make_changes(10, [7, 5, 1])
assert (7, 5, 1) == make_changes(13, [7, 5, 1])
assert (7, 7) == make_changes(14, [7, 5, 1])

!! **queens in peace, expandable(candidate, move)** 8.12 Given an n x n chessboard, write a program to place eight queens so that none of them share the same row, column, or diagonal -- Visualize

In [ ]:
def within_bounds_n_limits(queen_seq, r, c):  # peaceful at row r, column c?
  return all([
      c0 != c and r - r0 != abs(c - c0)  # r > r0; it processes rows [0, n).
      for r0, c0 in enumerate(queen_seq)
  ])

def queens_in_peace(n, queen_seq):  # n: number of queens to place in queen_seq.
  r = len(queen_seq)
  if r == n:
    yield tuple(queen_seq)
  else:
    for c in range(n):
      if within_bounds_n_limits(queen_seq, r, c):
        queen_seq.append(c)
        yield from queens_in_peace(n, queen_seq)
        queen_seq.pop()

assert ((1, 3, 0, 2), (2, 0, 3, 1)) == tuple(queens_in_peace(4, []))
expected = ((0,),), (), ()
assert expected == tuple(tuple(queens_in_peace(n, [])) for n in range(1, 4))

8.13 Given n boxes that cannot be rotated, but can only be stacked up, write a program to find the tallest possible stack, where the height of a stack is the sum of height of each box. Each box in the stack must be larger than the above it.



In [ ]:
def stack(boxes):
  @lru_cache(maxsize=None)
  def prog(n, L_limit=None):
    recap = lambda kv, k, v: (kv[0] + k, kv[1] + v)
    if n==0:
      return 0, ()
    else:
      box = boxes[n-1]
      cases = (
          prog(n-1, L_limit), recap(prog(n-1, box[0]), box[2], (n-1,))  #
          if L_limit is None or box[0] <= L_limit else (0, ()))
      return max(cases, key=lambda kv: kv[0])

  height, choices = prog(len(boxes))
  return height, choices

boxes = [  # sorted by W (width).
  (100, 30, 10), (200, 40, 10), (300, 50, 21),
  (10, 200, 30), (20, 300, 10), (30, 400, 20)
]  # yapf:disable
assert (60, (3, 4, 5)) == stack(boxes)

8.14 Given a boolean equation, write a program to count the number of ways to parenthesize the expression such that equation is true, e.g., INPUT: Expression: 1^0|0|1, Desired Result: false(0), OUTPUT: 2 ways: 1^((0|0)|1) and 1^(0|(0|1)).

In [ ]:
def order(operations, bit):
  n = len(operations) // 2  # n operations to parenthesis.
  if n<2:
    return 1 if bit == eval(operations) else 0
  else:
    m = 0
    for p in range(n):
      op, lhs, rhs = (operations[2*p+1], operations[0:2*p+1], operations[2*p+2:])
      lhs_0, lhs_1, rhs_0, rhs_1 = (
          order(e, bit) for e, bit in ((lhs, 0), (lhs, 1), (rhs, 0), (rhs, 1)))
      if op == '|':
        m += ((lhs_0*rhs_1 + lhs_1*rhs_0 + lhs_1*rhs_1) if bit==1 else
              (lhs_0 * rhs_0))
      if op == '&':
        m += ((lhs_1 * rhs_1) if bit==1 else
              (lhs_0*rhs_1 + lhs_1*rhs_0 + lhs_0*rhs_0))
      if op == '^':
        m += ((lhs_0*rhs_1 + lhs_1*rhs_0) if bit==1 else (lhs_0*rhs_0 + lhs_1*rhs_1))
    return m

assert 2 == order("1^0|0|1", 0)

In [ ]:
def interleave(L, R):
  if L == "" and R == "":
    yield ""
  elif L == "" or R == "":
    yield L + R
  else:
    yield from (L[0] + e for e in interleave(L[1:], R))
    yield from (R[0] + e for e in interleave(L, R[1:]))

interleaved = "ab12", "a1b2", "a12b", "1ab2", "1a2b", "12ab"
assert interleaved == tuple(interleave("ab", "12"))
assert 20 == len(tuple(interleave("abc", "123")))

!! **rotated, sorted integer cycles** 10.3 Given an array of sorted integers that has been rotated a few times, write a program to find an element in the array.

In [ ]:
def index_in_cycle(L, key, start=0, stop=None):
  if stop is None:
    stop = len(L)
  while stop - start>0:
    mid = (start+stop) // 2
    if L[mid] == key:
      return mid
    if L[mid] > L[stop-1]:
      if L[start] <= key < L[mid]:  # monotonicLlly rising.
        stop = mid
      else:
        start = mid+1
    else:
      if L[mid] < key <= L[stop-1]:  # monotonically rising.
        start = mid+1
      else:
        stop = mid

def min_in_cycle(L, start=0, stop=None):
  if stop is None:
    stop = len(L)
  if stop - start>1:
    if L[mid := (start+stop-1) // 2] > L[stop-1]:
      return min_in_cycle(L, mid+1, stop)
    else:  # RHS is monotonically rising.
      return min_in_cycle(L, start, mid+1)
  else:
    return start

deq = [
    deque([7, 8, 0, 1, 2, 3, 4, 5, 6]),  #
    deque([3, 4, 5, 6, 7, 8, 0, 1, 2])
]
for q in deq:
  for e in (0, 3, 5, 7):
    assert e == q[index_in_cycle(q, e)]

assert [0, 0, 1] == [min_in_cycle(L) for L in ([6], [6, 7], [7, 6])]
assert [2, 6] == [min_in_cycle(q) for q in deq]

def peak(L, start=0, stop=None):  # integers are rising and falling in L.
  if stop is None:
    stop = len(L)
  mid = (start+stop-1) // 2
  if L[mid-1] < L[mid] > L[mid+1]:
    return mid
  elif L[mid-1] < L[mid] < L[mid+1]:
    return peak(L, mid+1, stop)
  else:
    return peak(L, start, mid)

assert 6 == peak([1, 5, 7, 9, 15, 18, 21, 19, 14])

!! **sparse_bisect, sparse non-empty keys** 10.5 Given an array of sorted strings, which is interspersed with empty strings, write a method to find the location of a given string, e.g., find 'ball' in ['at', '', 'ball', '', '', 'car', '', 'dad', '', ''].

In [ ]:
def sparse_bisect(L, key, start=0, stop=None):
  if stop is None:
    stop = len(L)
  if stop - start>0:
    lo = hi = mid = (start+stop-1) // 2
    while not L[mid]:
      if lo < start and hi >= stop:
        return
      if lo >= start and L[lo]:
        mid = lo
      elif hi < stop and L[hi]:
        mid = hi
      else:
        lo, hi = lo-1, hi+1
    return (sparse_bisect(L, key, start, mid) if key < L[mid] else  #
            sparse_bisect(L, key, mid+1, stop) if key > L[mid] else mid)

assert 1 == sparse_bisect(["", "abc", "dos", "", "", "ijk", "xyz"], "abc")
assert 6 == sparse_bisect(["", "abc", "dos", "", "", "ijk", "xyz"], "xyz")

10.9 Given an NxM matrix in which each row and each column is sorted in ascending order, write a method to find an element. LC 240. Search a 2D matrix II.

In [ ]:
def find_in_grid(g, q, rows=None, cols=None):  # 3/4-section; not bisection.
  if rows is None:
    rows = (0, len(g))
  if cols is None:
    cols = (0, len(g[0]))
  if rows[1] > rows[0] and cols[1] > cols[0]:
    r, c = ((e[1] + e[0])//2 for e in (rows, cols))
    if q < g[r][c]:  # find in 3/4 of the section.
      return (find_in_grid(g, q, (rows[0], r), cols)
              or find_in_grid(g, q, (r, rows[1]), (cols[0], c)))
    elif q > g[r][c]:  # find in 3/4 of the section.
      return (find_in_grid(g, q, (r+1, rows[1]), cols)
              or find_in_grid(g, q, (rows[0], r+1), (c+1, cols[1])))
    else:
      return (r, c)

grid = [
    [ 1,  4,  7, 11, 15],
    [ 2,  5,  8, 12, 19],
    [ 3,  6,  9, 16, 22],
    [10, 13, 14, 17, 24],
    [18, 21, 23, 26, 30],
]  # yapf:disable
assert [(2, 1), (1, 2), None] == [find_in_grid(grid, q) for q in (6, 8, 20)]

In [ ]:
def slice_sorted_two(a1, a2, rng=None, rng1=None, rng2=None):
  rng1 = rng1 or [0, len(a1)]
  rng2 = rng2 or [0, len(a2)]
  m, n = rng1[1] - rng1[0], rng2[1] - rng2[0]
  if rng is None:
    k, z = divmod(m + n, 2)
    rng = [k, k+1 if z else k+2]
  if n==1:
    rng1[0] += rng[0]-1
    rng[0], rng[1] = 0, rng[1] - rng[0]
  elif m==1:
    rng2[0] += rng[0]-1
    rng[0], rng[1] = 0, rng[1] - rng[0]
  if rng[0]==0:
    res = []
    for _ in range(rng[0], rng[1]):
      if rng1[0] < len(a1) and rng2[0] < len(a2):
        if a1[rng1[0]] < a2[rng2[0]]:
          res.append(a1[rng1[0]])
          rng1[0] += 1
        else:
          res.append(a2[rng2[0]])
          rng2[0] += 1
      elif rng1[0] < len(a1):
        res.append(a1[rng1[0]])
        rng1[0] += 1
      elif rng2[0] < len(a2):
        res.append(a2[rng2[0]])
        rng2[0] += 1
    return res
  ext1 = rng[0] * m // (m+n)
  ext2 = rng[0] - ext1
  cmp0 = a1[rng1[0] + ext1] - a2[rng2[0] + ext2]
  if cmp0<0:
    rng[0], rng[1] = rng[0] - ext1, rng[1] - ext1
    rng1[0] = rng1[0] + ext1
    rng2[1] = rng2[0] + ext2
  else:
    rng[0], rng[1] = rng[0] - ext2, rng[1] - ext2
    rng1[0] = rng1[0] + ext2
    rng2[1] = rng2[0] + ext1
  return slice_sorted_two(a1, a2, rng, rng1, rng2)

assert ([2, 3, 4, 5, 11, 12, 13]  #
        == slice_sorted_two(list(range(1, 6)), list(range(11, 16)), [2, 9]))

!! **seekable binary search tree, track and rank, or track LE (lesser or same)**  
10.10 Design and implement a data structure and an algorithm that can track a stream of numbers, and tell the rank of a value x (the number of values less than or equal to x).  
4.11 Design a binary search tree capable of selecting a random node with equal probability. Operations such as inserting, deleting, finding, and randomly selecting a node should run in sublinear time. Seekable binary search tree has operations:
* bisect method yields bisecting tree nodes from the leaf to the root.
* getitem method returns the leaf node from the bisect, or none, if not found.
* setitem method get the leaf node from bisect, and makes a node of a new key.
* seek method returns the node, when `index` equals to the size of L sub-tree.

In [ ]:
class BTree:  # Design and implement a seekable binary search tree.
  def __init__(self, data=None, left=None, right=None, key=None):
    lr = (c if isinstance(c, (BTree, type(None))) else BTree(c) for c in (left, right))
    self.data, self.left, self.right = data, *lr
    self.key = data if key is None else key
    self._size = 1 + sum(c._size for c in (self.left, self.right) if c)
  #
  def bisect(self, key):
    if key < self.key and self.left:
      yield from self.left.bisect(key)
    elif key > self.key and self.right:
      yield from self.right.bisect(key)
    yield self
  #
  def __getitem__(self, key):
    if key == (tree := next(self.bisect(key))).key:
      return tree
  #
  def __setitem__(self, key, data):
    peeked = next(it := self.bisect(key))
    if key == peeked.key:
      peeked.data = data
    else:
      if key < peeked.key:
        peeked.left = BTree(data, key=key)
      else:
        peeked.right = BTree(data, key=key)
      for tree in chain([peeked], it):  # Recalculate _size in a bottom-up fashion.
        tree._size = 1 + sum(c._size for c in (tree.left, tree.right) if c)
  #
  def seek(self, index: int):
    L_size = self.left and self.left._size or 0
    if index < L_size:
      return self.left.seek(index) if self.left else None
    elif index > L_size:
      return self.right.seek(index -L_size -1) if self.right else None  # yapf:disable
    else:
      return self

# tree:   5
#       / . \
#      2  .  9
#       3 . 7
#            8
BTree.__repr__ = pp
b6 = BTree(5, BTree(2, None, 3), BTree(9, BTree(7, None, 8)))
b6_expected = (2, 3, 5, 7, 8, 9, None)
assert b6_expected == tuple((e := b6.seek(i)) and e.data for i in range(7))

In [ ]:
def track(tree, k):  # alternatively, track_LE (the lesser or the same)
  tree.data += 1
  if k < tree.key:
    if tree.left:
      track(tree.left, k)
    else:
      tree.left = BTree(1, key=k)
  elif k > tree.key:
    if tree.right:
      track(tree.right, k)
    else:
      tree.right = BTree(1, key=k)
  return tree

def rank(tree, k):  # rank: 1-based.
  if k <= tree.key:
    return rank(tree.left, k) if tree.left else 1
  assert k > tree.key
  L_rank = tree.data - (tree.right and tree.right.data or 0)
  R_rank = rank(tree.right, k) if tree.right else 1
  return L_rank + R_rank

# tree:     13(15)
#           /
#         4(8)
#       /      \
#    2(4)       7(4)
#    /  \       /  \
# 1(2) 3(2)   6(2) 9(2)
BTree.track, BTree.rank = track, rank
stream = (4, 4, 2, 2, 3, 3, 1, 1, 7, 7, 6, 6, 9, 9)
tr = reduce(lambda tr, e: tr.track(e), stream, BTree(1, key=13))
assert [1, 1, 3, 5, 7, 9, 9, 11, 13, 13, 15] == [rank(tr, e) for e in range(11)]

!! **interval tree, interval insert, inverval merge, interval extract** https://www.geeksforgeeks.org/interval-tree/

* Find all overlapping intervals given a query point/interval.
* Consolidate overlapping intervals given a list of intervals.
* Find the minimum set of points that cover all the intervals.
* Extract the largest contiguous interval given integer points.

In [ ]:
def overlap(intvl, query):  # returns an overlapping interval [left-, right-end].
  ol_intvl = max(intvl[0], query[0]), min(intvl[1], query[1])
  return ol_intvl if ol_intvl[0] <= ol_intvl[1] else None

class ITree:  # an itree has _max, besides an interval, and data.
  def __init__(self, intvl, left=None, right=None, data=None):
    self._max = max(chain([intvl[1]], (c._max for c in (left, right) if c)))
    self.intvl, self.left, self.right, self.data = intvl, left, right, data
  def __getitem__(self, intvl):
    if overlap(self.intvl, intvl):
      yield self
    if self.left and intvl[0] <= self.left._max:
      yield from self.left[intvl]
    if self.right and intvl[1] >= self.right.intvl[0]:
      yield from self.right[intvl]
  def __setitem__(self, intvl, data):
    if self.intvl == intvl:
      self.data = data
    elif intvl[0] <= self.intvl[0]:
      if self.left:
        self.left[intvl] = data
      else:
        self.left = ITree(intvl, data=data)
    else:
      if self.right:
        self.right[intvl] = data
      else:
        self.right = ITree(intvl, data=data)
    self._max = max(self._max, intvl[1])

Intvl = namedtuple("Intvl", "l_end, r_end")
Intvl.__abs__ = lambda self: abs(self.r_end+1 - self.l_end)

def insert_interval(seq, new_intvl):
  start = bisect.bisect_left(seq, new_intvl.l_end, key=lambda i: i.r_end)
  stop = bisect.bisect_right(seq, new_intvl.r_end, key=lambda i: i.l_end)
  if stop > start:
    l_end = min(new_intvl.l_end, seq[start].l_end)
    r_end = max(new_intvl.r_end, seq[stop-1].r_end)
    new_intvl = Intvl(l_end, r_end)
  seq[start:stop] = [new_intvl]
  return seq

def unify_intervals(seq):  # consolidate, merge intervals in L.
  it = iter(sorted(map(Intvl._make, seq)))  # iter in lexicographical order!
  unified = list(islice(it, 1))  # list(islice(it, 1)) vs. next(it, None)!
  for intvl in it:
    if intvl.l_end <= (u_r_end := unified[-1].r_end):
      unified[-1] = Intvl(unified[-1].l_end, max(u_r_end, intvl.r_end))
    else:
      unified.append(intvl)
  return unified

def fewest_points_covering_intervals(L):  # points that cover intervals in L.
  L = sorted(map(Intvl._make, L))
  points = []
  for intvl in L:
    if not points or points[-1] < intvl.l_end:
      points.append(intvl.r_end)
  return points

# https://leetcode.com/problems/longest-consecutive-sequence/
def extract_largest_contiguous_intervals(*L):  # for longest contiguous ranges.
  ranges = {}
  for point in set(L):  # same as dedupe(L).
    start, stop = point + ranges.pop(point, 0), point+1 + ranges.pop(point+1, 0)
    stride = stop - start
    ranges[start], ranges[stop] = stride, -stride
  return [(l_end, l_end + stride)
          for l_end, stride in maxima(ranges.items(), key=lambda e: e[1])]

assert [(7, 11)] == extract_largest_contiguous_intervals(3, 1, 9, 7, 2, 8, 10)
assert [(0, 3), (5, 13)] == unify_intervals([(0, 2), (1, 3), (5, 13), (9, 11)])
assert [(0, 13)] == unify_intervals([[0, 9], [1, 5], [3, 7], [9, 13]])
assert [(0, 0)] == unify_intervals([(0, 0)])
assert [] == unify_intervals([])
L = [Intvl(*e) for e in [(-3, -1), (2, 5), (7, 8)]]
assert [(-5, -4)] + L == insert_interval(L[:], Intvl(-5, -4))
assert L[:1] + [(0, 1)] + L[1:] == insert_interval(L[:], Intvl(0, 1))
assert [(-3, 0)] + L[1:] == insert_interval(L[:], Intvl(-2, 0))
assert L[:1] + [(1, 5)] + L[2:] == insert_interval(L[:], Intvl(1, 3))
assert [(-3, 5), (7, 8)] == insert_interval(L[:], Intvl(-2, 3))
assert L == insert_interval(L[:], Intvl(3, 4))
assert L + [(9, 9)] == insert_interval(L[:], Intvl(9, 9))
assert [3, 9] == fewest_points_covering_intervals([[0, 3], [2, 6], [3, 4], [6, 9]])

tree = ITree((15, 20))
for intvl in ((10, 25), (5, 20), (12, 15), (17, 19), (30, 40)):
  tree[intvl] = None
assert ((10, 25), (5, 20)) == tuple(e.intvl for e in tree[(4, 11)])
assert ((10, 25), (30, 40)) == tuple(e.intvl for e in tree[(21, 30)])

assert (2, 3) == overlap((1, 3), (2, 4))
assert (2, 3) == overlap((2, 4), (1, 3))
assert (2, 3) == overlap((1, 4), (2, 3))
assert (2, 3) == overlap((2, 3), (1, 4))
assert (3, 3) == overlap((2, 3), (3, 4))
assert overlap((1, 2), (3, 4)) is None

!! **KD tree, search closest, nearest** Find the 3 closest police stations near each branch office in a city.

In [ ]:
def kdtree_from_points(cls, points, axis=0, start=0, stop=None):
  if stop is None:
    stop = len(points)
  if stop > start:
    axis %= len(points[0])
    points[start:stop] = sorted(points[start:stop], key=lambda e: e[axis])
    mid = (start+stop-1) // 2  # mid becomes start, when stop-start-2.
    L = kdtree_from_points(cls, points, axis+1, start, mid)
    R = kdtree_from_points(cls, points, axis+1, mid+1, stop)
    return cls(points[mid], axis, L, R, mid)

class KdTree:
  @property
  def data(self):
    return self.point if self.data_ is None else self.data_
  def __init__(self, point, axis=0, left=None, right=None, data=None):
    self.point, self.axis, self.left, self.right, self.data_ = point, axis, left, right, data

def sq_distance(one, another):  # SED, squared euclidean distance.
  return sum((e - another[i])**2 for i, e in enumerate(one))

def search_kd(tree, query, n=1):
  def proc(tree, heap):
    if tree:
      heappush(heap, (-sq_distance(tree.point, query), tree.data))
      while len(heap) > n:
        heappop(heap)

      nearer, farther = tree.left, tree.right
      if (ax_diff := query[tree.axis] - tree.point[tree.axis])>0:
        nearer, farther = tree.right, tree.left
      proc(nearer, heap)
      if len(heap) < n or -heap[0][0] > ax_diff**len(tree.point):
        proc(farther, heap)
      return heap

  nearest_n = sorted(proc(tree, []), key=lambda e: -e[0])  # -e[0]: sq distance.
  return tuple((round(sqrt(-sq_d), 1), i) for sq_d, i in nearest_n)

KdTree.__repr__ = pp
KdTree.from_points = classmethod(kdtree_from_points)

points = [(-10.1, 10.1), (12.2, -5.2), (5.3, 38.3), (-179.9, 10.9), (0.0, 0.0)]
tree = KdTree.from_points(points)
nearest_3 = tuple((d, points[i]) for d, i in search_kd(tree, (1, 2), 3))
assert ((2.2, (0.0, 0.0)), (13.3, (12.2, -5.2)), (13.7, (-10.1, 10.1))) == nearest_3

16.1 Write a program to swap two numbers in-place without using additional variables.  
16.2 Write a program to find the frequency of occurrences of any given word in a book.  
16.3 Given two line segments, write a program to identify the point, where they cross each other.  
16.4 Write a program to test if a player has won a tic-tac-toe game.  
16.5 Write a method to compute the number of trailing zeros in n factorial.
```python
zeros = lambda n: (q := n // 5) + (zeros(q) if q > 4 else 0)  # quotient!
assert (0, 1, 4, 6, 18) == tuple(zeros(n) for n in (4, 5, 24, 25, 75))
```
16.6 Given two integer arrays, find the pair of integers (one from each array) with the smallest absolute difference. How about sorting and scanning two arrays?  
16.7 Write a program that finds the maximum of two numbers. Do not use if-else and comparisons.  

!! **Spell out an English number** 16.8 Spell out English words for numerical figures. Write "One Thousand Two Hundred Thirty Four" for 1234.

In [ ]:
digits = [None, "One", "Two", "Three", "Four", "Five", "Six", "Seven", "Eight", "Nine"]
teens = ["Ten", "Eleven", "Twelve", "Thirteen", "Fourteen", "Fifteen", "Sixteen", "Seventeen", "Eighteen", "Nineteen"]  # yapf:disable
tens = [None, None, "Twenty", "Thirty", "Forty", "Fifty", "Sixty", "Seventy", "Eighty", "Ninety"] # yapf:disable
bigs = [None, "Thousand", "Million"]

def read(d, x1000=0):  # yield from chain(["Negative"], read(-d, x1000))
  if d==0 and x1000==0:  # not to read 1000 as "Thousand Zero".
    yield "Zero"
  elif d:
    yield from read(d//1000, x1000+1)
    yield from read3d(d % 1000)
    if x1000:
      yield bigs[x1000]

def read3d(d):
  if d>=100:
    yield from chain([digits[d//100], "Hundred"], read3d(d % 100))
  elif d>=20:
    yield from chain([tens[d//10]], read3d(d % 10))
  elif d>=10:
    yield teens[d-10]
  elif d:
    yield digits[d]

assert "Zero" == "".join(read(0))
assert "Ten Million Two Hundred Twelve Thousand Eleven" == " ".join(read(10212011))
assert "Negative Twenty Million One Hundred One Thousand" == " ".join(read(-20101000))

16.9 Create functions to perform multiplication, subtraction, and division on integers. You can only use additions.

16.10 Given a list of people and their birth and death years, calculate the year with the maximum number of living individuals.



In [ ]:
# @title ##### 16.11 Given two different lengths of wooden planks, short and long, and a specific number of planks K, find all possible total lengths of a diving board that can be constructed using exactly K planks. Each plank used must be either short or long.
def diving_boards(k: int, short: int, long_: int):
  @lru_cache(maxsize=None)
  def prog(k_planks: int, s_planks, l_planks):
    if k_planks==0:
      yield 0
    else:
      yield from chain(
          (e + short for e in prog(k_planks-1, s_planks+1, l_planks)),
          (e + long_ for e in prog(k_planks-1, s_planks, l_planks+1)),
      )

  yield from prog(k, 0, 0)

assert [6, 7, 8, 9] == list(diving_boards(3, 2, 3))

16.12 Given an XML document and a mapping of XML tags to integer values, write a program that compresses an XML doc with encoding rules.
* Element: Tag Attributes END Children END
* Attribute: Tag Value
* END: 0
* Tag: A predefined integer mapping, e.g., family -> 1, person ->2, firstName -> 3, lastName -> 4, state
* Value: A string value


16.13 Given two squares on a 2D plane, find a line that would cut these two squares in half. Each square has top and bottom sides running parallel to the x-axis.  
16.14 Given a set of points on a 2D plane, find the line that passes through the maximum number of points.  

In [ ]:
# @title ##### 16.15 Imagine a secret code with four colors: Red (R), Yellow (Y), Green (G), and Blue (B). A game player tries to guess the secret code by picking four colors. Scores are given as a hit, if the color and the place both are right, or a pseudo-hit, if the color is right, but the place is not right.
def score(codes, guess):
  i_hits = {i for i, c, g in zip(range(4), codes, guess) if c == g}
  c_codes = Counter(c for i, c in enumerate(codes) if i not in i_hits)
  c_guess = Counter(g for i, g in enumerate(guess) if i not in i_hits)
  return len(i_hits), sum(min(h, c_codes.get(g, 0)) for g, h in c_guess.items())

score("RGBY", "GGRR")

!! **slice unsorted subarray, sum rainwater, outline, skyline**

16.16 Given an array, find a subarray that, when sorted, sorts the entire array.  
17.21: Given n non-negative integers representing an elevation map where the width of each bar is 1, compute how much water it is able to trap after raining. For example, given [0,1,0,2,1,0,1,3,2,1,2,1], return 6.

In [ ]:
def slice_unsorted_subarray(*L):  # scan forward, backward to see where to start, stop.
  def renumerate(it, stop=None):  # stop: defaults to the length of iterable.
    (R := list(it)).reverse()
    return zip(count(stop or len(R), -1), R)

  L_cummax = zip(L, accumulate(L, max))  # to see max_a: max ahead.
  stop = next((stop for stop, (e, max_a) in renumerate(L_cummax) if e < max_a), 0)
  L_cummin = zip(L, reversed(list(accumulate(reversed(L), min))))  # min_a: min ahead.
  start = next((start for start, (e, min_a) in enumerate(L_cummin) if e > min_a), 0)
  return slice(start, stop)

assert slice(0, 0) == slice_unsorted_subarray()
assert slice(0, 0) == slice_unsorted_subarray(1)
assert slice(0, 0) == slice_unsorted_subarray(1, 3)
assert slice(0, 2) == slice_unsorted_subarray(2, 1, 3)
assert slice(1, 4) == slice_unsorted_subarray(0, 2, 3, 1, 4)
assert slice(3, 5) == slice_unsorted_subarray(0, 1, 1, 2, 1)

In [ ]:
def produce_outline(blocks_by_L):  # to produce a skyline given building blocks.
  blocks_by_R = []  # k_block: right, block
  blocks_by_H = []  # k_block: -height, block
  exited = set()  # blocks exited.
  y_line = 0
  while blocks_by_L or blocks_by_R:
    cases = (
        blocks_by_L and blocks_by_L[0].key,  #
        blocks_by_R and blocks_by_R[0].key)
    x_line = min(filter(None, cases))  # also called x-location.
    while blocks_by_R and x_line == blocks_by_R[0].key:  # block exits
      exited.add(heappop(blocks_by_R).block)
    while blocks_by_L and x_line == blocks_by_L[0].key:  # block entries.
      _, block = blocks_by_L.popleft()
      heappush(blocks_by_R, kBlock(block.right, block))
      heappush(blocks_by_H, kBlock(-block.Ht, block))
    while blocks_by_H and blocks_by_H[0].block in exited:  # tidy up heights.
      exited.remove(heappop(blocks_by_H).block)
    if y_line != (h := -blocks_by_H[0].key if blocks_by_H else 0):
      yield (x_line, (y_line := h))  # yield a cross-point of x and y lines.

Block = namedtuple("Block", "left, Ht, right")
kBlock = namedtuple("kBlock", "key block")  # a sort key, a block.
buildings = ((1, 9, 3), (1, 11, 5), (2, 6, 7), (3, 13, 9), (12, 7, 16),  #
             (14, 3, 25), (19, 18, 22), (23, 13, 29), (24, 4, 28))
buildings = deque(kBlock(b[0], Block(*b)) for b in buildings)
expected = ((1, 11), (3, 13), (9, 0), (12, 7), (16, 3), (19, 18), (22, 3), (23, 13), (29, 0))  # yapf:disable
assert expected == tuple(produce_outline(buildings))

In [ ]:
# @title ##### 16.17 Given an array of integers (both positive and negative), write a program to find the max sum sub-array (contiguous sequence).

In [ ]:
# @title ##### 16.18 Pattern Matching : You are given two strings, pattern and value.The pattern string consists of just the letters aand b, describing a pattern within a string. For example, the string catcatgocatgomatches the pattern aabab(where catis aand gois b). It also matches patterns like a, ab, and b. Write a method to determine if value matches pattern.
def match(text: str, pattern: str):
  def prog(T, start, stop, **kwargs):
    if stop == start:
      if T == "":
        yield kwargs
    else:
      c = pattern[start]
      if lead := kwargs.get(c, None):
        if T.startswith(lead):
          yield from prog(T[len(lead):], start+1, stop, **kwargs)
      else:
        m = len(T)
        for k in range((m+1) // pattern[start:stop].count(c), 1, -1):
          lead, tail = T[:k], T[k:]
          yield from prog(tail, start+1, stop, **(kwargs | {c: lead}))

  yield from prog(text, 0, len(pattern))

# list(match("catcatgocatgo", "a"))
# list(match("catcatgocatgo", "ab"))
# list(match("catcatgocatgo", "aba"))
# list(match("catcatgocatgo", "aabab"))

16.19 Pond Sizes: You have an integer matrix representing a plot of land, where the value at that location
represents the height above sea level. A value of zero indicates water. A pond is a region of water
connected vertically, horizontally, or diagonally. The size of the pond is the total number of
connected water cells. Write a method to compute the sizes of all ponds in the matrix.
EXAMPLE
Input:
0 2 1 0
0 1 0 1
1 1 0 1
0 1 0 1
Output: 2, 4, 1 (in any order)

In [ ]:
# @title ##### 16.21 Given two integer arrays, find two integers (one from each array) that can be swapped to equalize their sums.
lhs, rhs = [1, 2, 3, 5], [4, 5, 6]
q, r = divmod(sum(lhs) - sum(rhs), 2)
assert r == 0
matcher, seen = {q+e for e in rhs}, set()
for e in lhs:
  if e in matcher and e not in seen:
    print((e, e-q))
    seen |= {e}

In [ ]:
# @title ##### 16.22 An ant starts on an infinite grid of alternating white and black squares, facing right. At each step:
# On a white square, it flips the color, turns 90° clockwise, and moves forward.
# On a black square, it flips the color, turns 90° counterclockwise, and moves forward.
# Write a program to simulate the ant's movement for K steps and print the final grid configuration.
# The grid structure must be designed by you. The input is the integer K, and the program should print the grid without returning anything.
# A possible method signature is void printKMoves(int K).

In [ ]:
# @title ##### 16.23 Given a function rand5() that generates a random number between 1 and 5 (inclusive), write a function that generates a random number between 1 and 7 (inclusive).
def rand5():
  import random
  return random.randrange(5) + 1

def rand7():
  rand25 = 22
  while rand25 > 21:  # [1, 21]
    rand25 = 5 * (rand5() - 1) + rand5()  # [1, 25]
  return rand25 % 7 + 1  # [1, 7].

from collections import Counter
c = Counter(rand7() for _ in range(7000))
assert all(900 < e < 1100 for e in c.values())

In [ ]:
# @title ##### 16.24 Find all pairs from integers, which sum to integer x.
def find_pair_sum(L, pair_sum):  # find distinct pairs; (1, 2), and (2, 1) are dups.
  matcher = set()
  seen = set()
  for e in L:
    if e in matcher and e not in seen:
      pair = tuple(sorted((e, pair_sum - e)))
      seen |= {*pair}
      yield pair
    else:
      matcher |= {pair_sum - e}

assert [(1, 2), (0, 3)] == list(find_pair_sum([1, 2, 3, 1, 0, -1], 3))

!! **LRU cache, circular buffer** 16.25 Design an LRU cache with a max size, that evicts the least recently used when full. You can assume the keys are integers and the values are strings.

In [ ]:
class DNode:
  def __init__(self, data, prev=None, next_=None):
    self.data, self.prev, self.next = data, prev, next_
  def __iter__(self):
    node = self
    while node:
      yield node
      node = node.next
  def __reversed__(self):
    node = self
    while node:
      yield node
      node = node.prev

class LRUCache:
  def __init__(self, capacity=1):
    self.capacity = capacity
    self.dmap = {}
    self.head = self.tail = None
  def __setitem__(self, k, v):
    if k in self.dmap:
      self.pop(self.dmap[k])
    self.append(DNode((k, v)))
    self.dmap[k] = self.tail
    while len(self.dmap) > self.capacity:
      del self.dmap[self.pop(self.head).data[0]]
  def __getitem__(self, k):
    if k in self.dmap:
      self.append(self.pop(self.dmap[k]))
      return self.tail.data[1]
  def append(self, node):  # wires up 4 links.
    node.next, node.prev = None, self.tail
    if self.tail:
      self.tail.next, self.tail = node, node
    else:  # was empty.
      self.head = self.tail = node
  def pop(self, node) -> DNode:  # wires up 2, 3, or 4 links
    if node.prev:
      node.prev.next = node.next
    else:  # was head.
      self.head = self.head.next
      self.head.prev = None
    if node.next:
      node.next.prev = node.prev
    else:  # was tail.
      self.tail = self.tail.prev
      self.tail.next = None
    return node

DNode.__repr__ = pp
lru = lambda lru: tuple(e.data for e in lru.head)
mru = lambda lru: tuple(e.data for e in reversed(lru.tail))
c = LRUCache(capacity=3)
c[1], c[2], c[3] = "a", "b", "c"
assert "a" == c[1]
assert ((2, "b"), (3, "c"), (1, "a")) == lru(c)
assert ((1, "a"), (3, "c"), (2, "b")) == mru(c)
assert "b" == c[2]
c[4], c[5] = "d", "e"
assert ((2, "b"), (4, "d"), (5, "e")) == lru(c)
assert ((5, "e"), (4, "d"), (2, "b")) == mru(c)

In [ ]:
|class CircularBuffer:  # http://www.programiz.com/python-programming/exceptions
  def __init__(self, capacity):  # 1 slot is reserved for simpler full, empty logic.
    self._L = [None] * (capacity+1)
    self._head = self._tail = 0
  def enq(self, v):
    if self.full():
      raise RuntimeError("This buffer is full.")
    self._L[self._tail] = v
    self._tail = (self._tail+1) % len(self._L)
    return self
  def deq(self):
    if self.empty():
      raise RuntimeError("This buffer is empty.")
    v = self._L[self._head]
    self._head = (self._head+1) % len(self._L)
    return v

CircularBuffer.empty = lambda self: self._head == self._tail
CircularBuffer.full = lambda self: self._head == (self._tail+1) % len(self._L)

buffer = CircularBuffer(3)
assert buffer.empty() and not buffer.full()
assert buffer.enq(10).enq(20).enq(30).full()
try:
  buffer.enq(40)
except RuntimeError:
  pass
assert [10, 20, 30] == [buffer.deq() for _ in range(3)]
assert buffer.empty()
try:
  buffer.deq()
except RuntimeError:
  pass

!! **consume, evaluate postfix (RPN), produce postfix (RPN)** 16.26 Evaluate an arithmetic expression with positive integers and the four basic operations.

In [ ]:
def evaluate_postfix(q):  # consumes a rpn Q: reverse polish notation.
  op_stack = []
  for e in q:
    print(f">> {op_stack=}")
    if e in op_prec:
      lhs, rhs = op_stack.pop(), op_stack.pop()
      print(f"?? {lhs}{e}{rhs}")  # BUG?
      e = eval(f"{lhs}{e}{rhs}")  # BUG?
    op_stack.append(e)
    print(f"<< {op_stack=}")
  return op_stack.pop()


In [ ]:
op_prec = {"*": 1, "/": 1, "+": 2, "-": 2, "(": 3}  # precedence

def produce_postfix(infix):  # produces a postfix, rpn, reverse polish notation.
  q, op_stack, digits = [], [], []
  for e in infix:
    if not e.isdigit() and digits:
      yield "".join(digits)
      digits.clear()
    if e.isdigit():
      digits.append(e)
    elif e == "(":
      op_stack.append(e)
    elif e == ")":
      while op_stack and op_stack[-1] != "(":
        yield op_stack.pop()
      op_stack.pop()  # Remove "("
    elif e in op_prec:
      while op_stack and op_prec.get(op_stack[-1], float('-inf')) <= op_prec[e]:
        yield op_stack.pop()
      op_stack.append(e)

  if digits:
    yield "".join(digits)
  yield from reversed(op_stack)

def evaluate_postfix(q):  # consumes a rpn Q: reverse polish notation.
  op_stack = []
  for e in q:
    if e in op_prec:
      lhs, rhs = op_stack.pop(), op_stack.pop()
      e = eval(f"{lhs}{e}{rhs}")
    op_stack.append(e)
  return op_stack.pop()

infix = "2*3+14+(2*3+4)*2"
assert "23*14+23*4+2*+" == "".join(produce_postfix(infix))
assert 40 == evaluate_postfix(produce_postfix(infix))

In [ ]:
# @title ##### 18.1 Write a functions that adds two numbers. You should not use the addition (+) arithmetic operator.
def sum_(a, b):  # a: ones, b: carries
  return sum_(a ^ b, (a & b)<<1) if b>0 else a

assert 10 == sum_(7, 3)

!! **random shuffle, reservoir_samples, weighted choice**

18.2 Write a function to shuffle a deck of cards. Each of the 52! permutations of the deck has to be equally probable.  
18.3 Write a function to randomly sample m integers out of n integers. Each element must have equal probability of being chosen.

In [ ]:
class Deck:
  def __init__(self, n):
    self.n_cards = n
    self.swapped = {}
  def cards(self):
    while self.n_cards>0:
      r = random.randrange(self.n_cards)
      yield self.swapped.get(r, r)
      self.swapped[r] = self.swapped.pop(self.n_cards-1, self.n_cards-1)
      self.n_cards -= 1

def fisher_yates_shuffle(L):
  n = len(L)
  for i in range(n-1):
    j = i + random.randrange(n - i)  # j is in [i, n-1] from i + [0, n-i-1].
    L[j], L[i] = L[i], L[j]
  return L

def reservoir_samples(iterable, k):
  samples = []
  count = 0
  for e in iterable:
    count += 1
    if len(samples) < k:
      samples.append(e)
    elif (i := random.randrange(count)) < k:
      samples[i] = e
  return samples

def weighted_choice(weights):
  rand = random.randrange((cumsum := tuple(accumulate(weights)))[-1])
  return next(i for i, e in enumerate(cumsum) if rand < e)

assert list(range(9)) == sorted(Deck(9).cards())
assert 3 == len(reservoir_samples(iter(range(100)), 3))
weights = [20, 30, 50]
c = Counter(weighted_choice(weights) for _ in range(1000))
sorted(c.items(), key=lambda k: c[k])

In [ ]:
# @title ##### 17.5 Find the longest subarray that has an equal number of letters and digits.
def longest_subarray_with_alpha_nums_balanced(s):
  cumsum = accumulate(([-1, 1][c.isalpha()] for c in s), initial=0)
  maxima, max_key = [], 0
  indices = {}  # indices keyed by a cunsum, whenever you see the same cumsum.
  for stop, sum_ in enumerate(cumsum, 0):
    if (start := indices.get(sum_, -1)) > -1:
      if (key := stop - indices[sum_]) > max_key:
        maxima, max_key = [(start, stop)], key
      elif key == max_key:
        maxima.append((start, stop))
    else:
      indices[sum_] = stop
  return maxima

assert [(0, 4), (3, 7)] == longest_subarray_with_alpha_nums_balanced("a12b55c")

In [ ]:
# @title ##### 17.6 Write a function to count the number of 2s between O and n.
def count_2s_upto_e(b):  # 1 up to E+1, 20 up to E+2, 300 up to E+3.
  return b * 10 ** (b - 1)

def count_2s_upto(n):
  count, e, p = 0, 0, n  # preceding(p) precedes number(n).
  while p > 0:
    d = p % 10  # we look at a digit (rE+e) in each iteration.
    count += d * count_2s_upto_e(e)
    if d > 3:
      count += 10**e  # e.g., count += 100 for [200, 299], when p >299 and e=2
    if d == 2:
      count += n % 10**e + 1  # e.g., count += 56 for [200, 255], when n: x255, e: 2.
    p //= 10
    e += 1
  return count

assert 1 == count_2s_upto(9)
assert 20 == count_2s_upto(99)
assert 300 == count_2s_upto(999)
assert 3059 == count_2s_upto(6789)

In [ ]:
# @title ##### 17.7 Each year, the government releases a list of the 10,000 most common baby names and their frequencies. Some names, like "John" and "Jon," are variations of the same name but are listed separately. Given two lists—one with names and frequencies, and another with equivalent name pairs—design an algorithm to consolidate frequencies for equivalent names. If "John" is equivalent to "Jon," and "Jon" to "Johnny," all three are treated as equivalent (transitive and symmetric relationship). The final list should group equivalent names and use any as the primary name.
def consolidate_baby_names(counter, synonyms):
  id_map = {}
  for lhs, rhs in synonyms:
    lhs_id, rhs_id = id_map.setdefault(lhs, lhs), id_map.setdefault(rhs, rhs)
    if lhs_id != rhs_id:
      id_ = ":".join(sorted(lhs_id.split(":") + rhs_id.split(":")))
      for name in id_.split(":"):
        id_map[name] = id_
      lhs_count, rhs_count = counter.pop(lhs_id, 0), counter.pop(rhs_id, 0)
      counter[id_] = lhs_count + rhs_count
  return counter

counter = dict(john=15, jon=12, chris=13, kris=4, christopher=19)
synonyms = (("jon", "john"), ("chris", "kris"), ("chris", "christopher"))
consolidate_baby_names(counter, synonyms)

In [ ]:
# @title ##### 17.8 Write a program to design a circus of the largest tower of people standing atop one another's shoulders. For practical and aesthetic reasons, each person must be both shorter and lighter than the person below him or her

In [ ]:
# @title ##### 17.9 Find the k-th number such that the only prime factors are 3, 5, and 7.
def produce_prime_multiples(primes):  # 2, 3, 5
  queues = [deque([p]) for p in primes]
  while True:
    yield (x := min(queue[0] for queue in queues))
    for i, queue in enumerate(queues):
      if x == queue[0]:
        queue.popleft()
      queue.append(x * primes[i])

assert (3, 5, 7) == tuple(islice(produce_prime_multiples((3, 5, 7)), 3))
assert (9, 15, 21, 25) == tuple(islice(produce_prime_multiples((3, 5, 7)), 3, 7))

!! **boyer_moore_majority** 17.10 Given an array of positive integers, determine the majority, if any. A majority is an element that occurs more than half the times in the array.

In [ ]:
def boyer_moore_majority(L):
  candidate, votes = None, 0
  for e in L:
    if votes==0:
      candidate, votes = e, 1
    elif e == candidate:
      votes += 1
    else:
      votes -= 1
  if L.count(candidate) > len(L) // 2:
    return candidate

cases = ([], [1], [1, 1, 2, 3], [1, 2, 1], [1, 2, 1, 2, 1])
assert (None, 1, None, 1, 1) == tuple(map(boyer_moore_majority, cases))

!! **shortest text snippet, shortest superset subarray, longest unique subarray**

17.11 Find the shortest text snippet that contains K words out of a large text.  
17.18 Find the shortest superset subarray that contains another shorter array -- https://leetcode.com/problems/minimum-window-substring

In [ ]:
def shortest_text_snippet(k_indices):  # O(L *logK), L: length of k indices.
  # positions, e.g., [[0, 89, 130], [95, 123, 177, 199], [70, 105, 117]]
  k_indices = [deque(e) for e in k_indices]
  min_window = window = [e.popleft() for e in k_indices]  # [0, 95, 70]
  heapify(heap := [(index, k) for k, index in enumerate(window)])
  while k_indices[heap[0][1]]:  # k_indices[2] becomes empty after 117.
    _, k = heappop(heap)
    window[k] = k_indices[k].popleft()
    min_window = min(min_window, window, key=lambda w: max(w) - min(w))
    heappush(heap, (window[k], k))
  return min_window

k_word_indices = [[0, 89, 130], [95, 123, 177, 199], [70, 105, 117]]
assert [130, 123, 117] == shortest_text_snippet(k_word_indices)

In [ ]:
# @title ##### 17.12 Write a program to convert a binary search tree to a doubly linked list. Keep the values in order while converting in-place.
def dlist_from_bst(tree):  # returns tuple(head, tail).
  it = iterate(tree)
  head = pred = next(it)
  for tree in it:
    tree.left, pred.right = pred, tree
    pred = tree
  return head, tree

def iterate_east(dlist):
  while dlist:
    yield dlist
    dlist = dlist.right

def iterate_west(dlist):
  while dlist:
    yield dlist
    dlist = dlist.left

head, tail = dlist_from_bst(BTree.from_values(range(10)))
head, tail
assert (0, 1, 2, 3, 4, 5, 6, 7, 8, 9) == tuple(e.data for e in iterate_east(head))
assert (9, 8, 7, 6, 5, 4, 3, 2, 1, 0) == tuple(e.data for e in iterate_west(tail))

In [ ]:
# @title ##### 4.3 Given a binary tree, design an algorithm to creates a linked list of all the nodes at each depth.
# If you have a tree with depth D, you will have D linked lists.
def dlist_from_binary_tree(tree):  # a tree of depth d.
  q_next = deque([tree])
  while q_next:
    q, q_next = q_next, deque()
    for i, e in enumerate(q):
      for c in (e.left, e.right):
        if c:
          q_next.append(c)
      e.left = None if i==0 else q[i-1]
      e.right = q[i+1] if i < len(q)-1 else None
    yield q[0]

t10 = BTree.from_values(range(10))
assert ([[4], [1, 7], [0, 2, 5, 8], [3, 6, 9]]  #
        == [[e.data for e in iterate_east(L)] for L in dlist_from_binary_tree(t10)])

In [ ]:
# @title ##### 17.13 Given a text with all spaces, punctuation, and capitalization removed, e.g., "iresetthecomputeri tstilldidntboot" from "I reset the computer. It still didn’t boot!". Respace the text while keeping most letters recognizable. Don't worry about capitalization or punctuation for now.
def re_space(text, vocas):
  from functools import lru_cache
  vocas = dict.fromkeys(sorted(vocas, key=len, reverse=True))
  recap = lambda kv, k, v: (kv[0] + k, kv[1] + v)  # recap of (minutes, sequence).
  oov_loss = lambda n, m: 0 if text[n:m] in vocas else m - n  # out-of-voca. loss.

  @lru_cache(maxsize=None)
  def prog(m):
    if m>0:
      m_cases = [recap(prog(n), oov_loss(n, m), (m,)) for n in range(m-1, -1, -1)]
      return min(m_cases, key=lambda e: e[0])
    else:
      return (0, ())

  rubbish, stops = prog(len(text))
  compose = lambda c: (text[i>0 and c[i-1]:stop] for i, stop in enumerate(c))
  return rubbish, " ".join(compose(stops))

vocas = ["you", "reset", "the", "computer", "still", "didnt", "boot"]
text = "youresetthecomputeritstilldidntboot"
assert (2, "you reset the computer i t still didnt boot") == re_space(text, vocas)

In [ ]:
# @title ##### 17.14 Write a program to find the smallest million numbers in billion numbers. Your computer can hold all one billion numbers.
# http://en.wikipedia.org/wiki/Selection_algorithm#Optimised_sorting_algorithms
def quickfind_k(L, k=0, start=0, stop=None, key=lambda e: e, sort=False):
  if stop is None:
    stop = len(L)
  if stop - start>1:
    pivot = partition(L, start, stop, key)
    if k < pivot or sort:
      quickfind_k(L, k, start, pivot, key, sort)
    if pivot < k:
      quickfind_k(L, k, pivot+1, stop, key, sort)
  return L

def partition(L, start, stop, key=lambda e: e):
  pivot = start + random.randrange(stop - start)
  L[pivot], L[start] = L[start], L[pivot]
  pivot = start
  for i in range(start+1, stop):
    if key(L[i]) < key(L[start]):
      pivot += 1
      L[pivot], L[i] = L[i], L[pivot]
  L[pivot], L[start] = L[start], L[pivot]
  return pivot

quicksort_k = partial(quickfind_k, sort=True)
assert [0, 1] == quickfind_k([9, 8, 7, 6, 5, 4, 3, 2, 1, 0], 1)[:2]
assert 2 == quickfind_k([9, 8, 7, 6, 5, 4, 3, 2, 1, 0], 2)[2]
assert 3 == quickfind_k([9, 8, 7, 6, 5, 4, 3, 2, 1, 0], 3)[3]

for _ in range(1000):
  L = [9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
  # fisher_yates_shuffle(L)
  L = quicksort_k(L, 3)
  if [0, 1, 2, 3] != L[:4]:
    print(f"{L=}")

In [ ]:
# @title ##### 17.15 Longest Word: Given a list of words, write a program to find the longest word made of other words in the list.
def longest_composition(L):
  vocas = dict.fromkeys(sorted(L, key=len, reverse=True))
  @lru_cache(maxsize=None)
  def prog(word, m):
    if m > 0:
      n = next((n for n in range(m - 1, 0, -1) if word[n:m] in vocas), None)
      if n:
        if word[:n] in vocas:
          yield (n, m)
        else:
          yield from (e + (m,) for e in prog(word, n))
    else:
      yield ()
  compose = lambda c: [word[i > 0 and c[i - 1] : stop] for i, stop in enumerate(c)]
  for word in vocas:
    yield from (compose(stops) for stops in prog(word, len(word)))

L =  ["cat", "cats", "catdogcats", "catcats", "dog", "dogcatsdog", "ratcatdogcats"]  # fmt: skip
assert ["cat", "dog", "cats"] == next(longest_composition(L), None)

In [ ]:
# @title ##### 17.16 A popular masseuse receives a sequence of back-to-back appointment requests and is debating which ones to accept. She needs a 15-minute break between appointments and therefore she cannot accept any adjacent requests. Given a sequence of back-to-back appointment requests (all multiples of 15 minutes, none overlap, and none can be moved), find the optimal (highest total booked minutes) set the masseuse can honor. Return the number of minutes.
def masseuse(L):  # L: massage appointmts in minutes.
  recap = lambda kv, k, v: (kv[0] + k, kv[1] + v)  # recap of (minutes, sequence).
  def prog(n):  # n appts. to process.
    if n>0:
      key, value = L[n-1], (n-1,)  # key: minutes, value: seq. of decisions.
      cases = (recap(prog(n-2), key, value), recap(prog(n-1), 0, ()))
      return max(cases, key=lambda e: e[0])
    else:  # no appts. to process.
      return 0, ()

  return prog(len(L))

L = [30, 15, 60, 75, 45, 15, 15, 45]  # a list of appointmts.
minutes, indices = masseuse(L)
assert (180, (30, 60, 45, 45)) == (minutes, tuple(L[i] for i in indices))

!! **rabin karp's string match in O(m+n)** Find all indices of a pattern in a long text, "ana" in "banana anaconda eats".

In [ ]:
# rabin karp's string match in O(m+n), not naive O(m x n) -- pattern(m), text(n).
def rabin_karp(pattern, text):
  def _hash(text, start, stop, hash=None):  # O(m), or O(1).
    if hash is None:
      return reduce(lambda h, c: h*31 + c, text[start:stop], 0)  # O(m).

    hash = (hash<<5) - hash
    new, old = text[stop-1], text[start-1]  # new and old character code points.
    return hash + new - old * 31**(stop - start)  # O(1).

  (pttn, m), (text, n) = ((tuple(map(ord, e)), len(e)) for e in (pattern, text))
  hash_p = _hash(pttn, 0, m)
  hash_t = None  # e.g., hash(text, 0, m)
  for i_text in range(n - m+1):
    hash_t = _hash(text, i_text, i_text + m, hash_t)
    if hash_p == hash_t and pttn == text[i_text:i_text + m]:
      yield i_text

assert (1, 3, 10) == tuple(rabin_karp("ana", "banana an anaconda eats."))

In [ ]:
# @title ##### 17.17 Given a text T and a set of queries Q, write a program to search T for each query in Q.
# Related: the longest common substrings to a set of queries Q.
class Trie:
  def __init__(self, value=None, **_):
    self.value, self.children = value, defaultdict(lambda: Trie())
  def __setitem__(self, key, child):
    if len(key)>1:
      self.children[key[0]][key[1:]] = child
    else:
      self.children[key[0]] = child if isinstance(child, Trie) else Trie(child)
  def __getitem__(self, key):
    return (c := self.children.get(key[0], None)) and c[key[1:]] if key else self
  def values(self):
    if self.value is not None:
      yield self.value
    for child in self.children.values():
      yield from child.values()

Trie.__repr__ = pp
s, suffs = "bananas", Trie()  # a suffix tree is a trie of all the suffixes.
for i, _ in enumerate(s):
  suffs[s[i:]] = i
#
L = "ba na nas s bas".split(" ")
expected = [[0], [2, 4], [4], [6], None]
assert expected == [(t := suffs[q]) and list(t.values()) for q in L]
# Trie for autocomplete
trie = Trie()
vocabs = ["brace", "brazil", "bread", "brew", "brag"]
for i, e in enumerate(vocabs):
  trie[e] = i  # or Trie(i).
#
assert (0, 1, 4) == tuple(trie["bra"].values())  # see the words with prefix `bra`
assert (2, 3) == tuple(trie["bre"].values())  # see the words with prefix `bre`.

17.19 Given an integer array from 1 to N appearing exactly once,
except for one number that is missing. How can you find the missing number in O(N) time and 0(1) space? What if there were two numbers missing?

17.20 Write a program that can quickly answer a median value, while random numbers are being generated and offered (a median bag).

17.22 Given two words of equal length that are in a dictionary, write a method to transform one word into another word by changing only one letter at a time. The new word you get in each step must be in the dictionary.

!! **maxsum submatrix, maxsum subarray, largest subsquare**

17.24 Given a square matrix of integers, write a program to find a sub-matrix with the largest sum.

17.23 Given a square matrix of black and white cells, write a program to find the maximum sub-square such that all four borders are filled with black pixels.



In [ ]:
def slice_maxsum_submatrix(g):
  cumsum_v = (accumulate(c, initial=0) for c in zip(*g))
  cumsum = tuple(tuple(accumulate(r, initial=0)) for r in zip(*cumsum_v))
  sum_2d = lambda start_r, start_c, stop_r, stop_c: (
      cumsum[stop_r][stop_c] - cumsum[stop_r][start_c] - cumsum[start_r][stop_c] +
      cumsum[start_r][start_c])

  # yield (0, (0, 0, 0, 0))
  m, n = len(g), len(g[0])  # this loop runs in O(m x n) time given an m x n graph.
  for stop_r in range(1, m+1):
    for start_r in range(stop_r):
      for stop_c in range(1, n+1):
        for start_c in range(stop_c):
          cut = start_r, start_c, stop_r, stop_c
          yield sum_2d(*cut), cut

# Kadane's algorithm http://en.wikipedia.org/wiki/maximum_subarray_problem
def slice_maxsum_subarray(L):  # returns (maxsum, (start, stop)).
  yield (sum_ := 0), (0, 0)
  for stop, e in enumerate(L, 1):
    if sum_>0:
      sum_ = sum_ + e  # keep the sum running.
    else:  # start over the sum running .
      sum_, start = e, stop-1
    yield sum_, (start, stop)

g = [  # 4 x 3 matrix
    [ 1,  0, 1],
    [ 0, -1, 0],
    [ 1,  0, 1],
    [-5,  2, 5]
]  # yapf:disable
assert (5, (1, 8)) == max(slice_maxsum_subarray([-2, 1, 3, -3, 4, -2, -1, 3]))
assert (9, (0, 3)) == max(slice_maxsum_subarray([9, -3, 3]))
assert (9, (2, 3)) == max(slice_maxsum_subarray([3, -3, 9]))
assert (0, (0, 0)) == max(slice_maxsum_subarray([-9]))
assert (0, (0, 1)) == max(slice_maxsum_subarray([0]))
assert (8, (2, 1, 4, 3)) == max(slice_maxsum_submatrix(g))
assert (3, (0, 0, 1, 2)) == max(slice_maxsum_submatrix([
    [ 1,  2, -1],
    [-3, -1, -4],
    [ 1, -5,  2]
]))  # yapf:disable

17.25 Given millions of words, write a program to create the largest possible rectangle of letters such that every row forms a word (reading left to right) and every column forms a word (reading top to bottom).



In [ ]:
def groupby(iterable, key=None, unique=False):
  d = {}
  if key is None:
    key = lambda e: e
  for e in iterable:
    k = key(e)
    if k in d:
      if unique:
        d[k].add(e)
      else:
        d[k].append(e)
    else:
      d[k] = set((e,)) if unique else [e]
  return d

def permutate(a, k=None, i=0):  # places values at indices from i to k-1.
  if k is None:
    k = len(a)
  if i == k:
    yield a[:k]
  else:
    for j in {a[j]: j for j in range(i, len(a))}.values():
      a[i], a[j] = a[j], a[i]
      yield from permutate(a, k, i + 1)
      a[i], a[j] = a[j], a[i]

assert {0: [2, 2], 1: [1, 3, 3, 3]} == groupby([1, 2, 2, 3, 3, 3], lambda e: e % 2)
assert {3: {"abc", "def"}, 4: {"wxyz"}} == groupby(["abc", "def", "wxyz"], len, True)
assert [[1, 1], [1, 2], [2, 1]] == sorted(permutate([1, 1, 2], 2))

def rectangle(words):
  d = groupby(words, len, unique=True)
  n = max(d.keys())
  area = n**2
  for a in range(area, 0, -1):
    for w in (w for w in range(n, 0, -1) if 0 == a % w):
      h = a // w  # height is area divided by width
      if w >= h and d.get(w) and len(d[w]) >= h:
        for p in permutate(list(d[w]), h):
          if all(
            ("".join((p[r][c] for r in range(h))) in d[h] for c in range(w))
          ):
            return p

words = set("a cat ate tea strafer taeniae resters antiflu fiefdom earlobe resumes schools coconut acacias oracle largest fingernails".split(" "))  # fmt: skip
assert 7 == len(rectangle(words))

17.26 Sparse Similarity: The similarity of two documents (each with distinct words) is defined to be the size of the intersection divided by the size of the union. For example, if the documents consist of integers, the similarity of {1, 5, 3} and { 1, 7, 2, 3} is 0. 4, because the intersection has size 2 and the union has size 5. We have a long list of documents (with distinct values and each with an associated ID) where the similarity is believed to be "sparse:'That is, any two arbitrarily selected documents are very likely to have similarity O. Design an algorithm that returns a list of pairs of document IDs and the associated similarity. Print only the pairs with similarity greater than 0. Empty documents should not be printed at all. For simplicity, you may assume each document is represented as an array of distinct integers.  
EXAMPLE nput:
* 13: {14, 15, 100, 9, 3}
* 16: {32, 1, 9, 3, 5}
* 19: {15, 29, 2, 6, 8, 7}
* 24: {7, 10}

ID1 | ID2 | SIMILARITY
-- | -- | --
13 | 19 | 0.1
13 | 16 | 0.25
19 | 24 | 0.143
